In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-energy-behavior-of-prosumers/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv
/kaggle/input/predict-energy-behavior-of-prosumers/public_timeseries_testing_util.py
/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/county_id_to_name_map.json
/kaggle/input/predict-energy-behavior-of-prosumers/train.csv
/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/sample_submission.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/electricity

In [2]:
# CORE DATA MANIPULATION
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr, spearmanr
import math

# MACHINE LEARNING - SCIKIT-LEARN
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, RFE, RFECV
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, roc_auc_score, roc_curve, auc
from sklearn.pipeline import Pipeline

# ADVANCED ML LIBRARIES
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor

# VISUALIZATION
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo

# CONFIGURATION FOR PLOTS
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline


# DEEP LEARNING 
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv2D, MaxPooling2D, Flatten, BatchNormalization
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    print("TensorFlow disponible")
except ImportError:
    print("TensorFlow no disponible en este entorno")


# UTILITIES
import os
import sys
import warnings
import itertools
from datetime import datetime, timedelta
import time
from collections import Counter
import pickle
import joblib

# JUPYTER SPECIFIC
from IPython.display import display, HTML
from tqdm.notebook import tqdm

# SUPPRESS WARNINGS
warnings.filterwarnings('ignore')

# PANDAS CONFIGURATION
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# NUMPY CONFIGURATION
np.random.seed(42)

# PLOTLY CONFIGURATION
pyo.init_notebook_mode(connected=True)

print("✅ Todas las librerías importadas correctamente!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"🤖 Scikit-learn version: {__import__('sklearn').__version__}")
print(f"📈 Matplotlib version: {__import__('matplotlib').__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")

# ============================================================================
# FUNCIONES ÚTILES ADICIONALES
# ============================================================================

def quick_info(df):
    """Información rápida del dataset"""
    print(f"📊 Dataset Shape: {df.shape}")
    print(f"🔢 Columnas numéricas: {df.select_dtypes(include=[np.number]).columns.tolist()}")
    print(f"📝 Columnas categóricas: {df.select_dtypes(include=['object']).columns.tolist()}")
    print(f"❌ Valores nulos por columna:")
    print(df.isnull().sum()[df.isnull().sum() > 0])

def plot_missing_values(df):
    """Visualizar valores faltantes"""
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if len(missing) > 0:
        plt.figure(figsize=(10, 6))
        missing.plot(kind='bar')
        plt.title('Valores Faltantes por Columna')
        plt.ylabel('Cantidad')
        plt.xticks(rotation=45)
        plt.show()
    else:
        print("✅ No hay valores faltantes en el dataset")


2025-09-17 20:06:24.781159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758139585.080430     227 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758139585.165584     227 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow disponible


✅ Todas las librerías importadas correctamente!
📊 Pandas version: 2.2.3
🔢 NumPy version: 1.26.4
🤖 Scikit-learn version: 1.2.2
📈 Matplotlib version: 3.7.2
🎨 Seaborn version: 0.12.2


In [3]:
#competencia https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/overview

# Files
# train.csv

# county - An ID code for the county.
# is_business - Boolean for whether or not the prosumer is a business.
# product_type - ID code with the following mapping of codes to contract types: {0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}.
# target - The consumption or production amount for the relevant segment for the hour. The segments are defined by the county, is_business, and product_type.
# is_consumption - Boolean for whether or not this row's target is consumption or production.
# datetime - The Estonian time in EET (UTC+2) / EEST (UTC+3). It describes the start of the 1-hour period on which target is given.
# data_block_id - All rows sharing the same data_block_id will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather data_block_id for predictins made on October 31st is 100 then the historic weather data_block_id for October 31st will be 101 as the historic weather data is only actually available the next day.
# row_id - A unique identifier for the row.
# prediction_unit_id - A unique identifier for the county, is_business, and product_type combination. New prediction units can appear or disappear in the test set.
# gas_prices.csv

# origin_date - The date when the day-ahead prices became available.
# forecast_date - The date when the forecast prices should be relevant.
# [lowest/highest]_price_per_mwh - The lowest/highest price of natural gas that on the day ahead market that trading day, in Euros per megawatt hour equivalent.
# data_block_id
# client.csv

# product_type
# county - An ID code for the county. See county_id_to_name_map.json for the mapping of ID codes to county names.
# eic_count - The aggregated number of consumption points (EICs - European Identifier Code).
# installed_capacity - Installed photovoltaic solar panel capacity in kilowatts.
# is_business - Boolean for whether or not the prosumer is a business.
# date
# data_block_id
# electricity_prices.csv

# origin_date
# forecast_date - Represents the start of the 1-hour period when the price is valid
# euros_per_mwh - The price of electricity on the day ahead markets in euros per megawatt hour.
# data_block_id
# forecast_weather.csv Weather forecasts that would have been available at prediction time. Sourced from the European Centre for Medium-Range Weather Forecasts.

# [latitude/longitude] - The coordinates of the weather forecast.
# origin_datetime - The timestamp of when the forecast was generated.
# hours_ahead - The number of hours between the forecast generation and the forecast weather. Each forecast covers 48 hours in total.
# temperature - The air temperature at 2 meters above ground in degrees Celsius. Estimated for the end of the 1-hour period.
# dewpoint - The dew point temperature at 2 meters above ground in degrees Celsius. Estimated for the end of the 1-hour period.
# cloudcover_[low/mid/high/total] - The percentage of the sky covered by clouds in the following altitude bands: 0-2 km, 2-6, 6+, and total. Estimated for the end of the 1-hour period.
# 10_metre_[u/v]_wind_component - The [eastward/northward] component of wind speed measured 10 meters above surface in meters per second. Estimated for the end of the 1-hour period.
# data_block_id
# forecast_datetime - The timestamp of the predicted weather. Generated from origin_datetime plus hours_ahead. This represents the start of the 1-hour period for which weather data are forecasted.
# direct_solar_radiation - The direct solar radiation reaching the surface on a plane perpendicular to the direction of the Sun accumulated during the hour, in watt-hours per square meter.
# surface_solar_radiation_downwards - The solar radiation, both direct and diffuse, that reaches a horizontal plane at the surface of the Earth, accumulated during the hour, in watt-hours per square meter.
# snowfall - Snowfall over hour in units of meters of water equivalent.
# total_precipitation - The accumulated liquid, comprising rain and snow that falls on Earth's surface over the described hour, in units of meters.
# historical_weather.csv Historic weather data.

# datetime - This represents the start of the 1-hour period for which weather data are measured.
# temperature - Measured at the end of the 1-hour period.
# dewpoint - Measured at the end of the 1-hour period.
# rain - Different from the forecast conventions. The rain from large scale weather systems of the hour in millimeters.
# snowfall - Different from the forecast conventions. Snowfall over the hour in centimeters.
# surface_pressure - The air pressure at surface in hectopascals.
# cloudcover_[low/mid/high/total] - Different from the forecast conventions. Cloud cover at 0-3 km, 3-8, 8+, and total.
# windspeed_10m - Different from the forecast conventions. The wind speed at 10 meters above ground in meters per second.
# winddirection_10m - Different from the forecast conventions. The wind direction at 10 meters above ground in degrees.
# shortwave_radiation - Different from the forecast conventions. The global horizontal irradiation in watt-hours per square meter.
# direct_solar_radiation
# diffuse_radiation - Different from the forecast conventions. The diffuse solar irradiation in watt-hours per square meter.
# [latitude/longitude] - The coordinates of the weather station.
# data_block_id

In [4]:
#modelos a correr: NN por ser forecasting y algun xgb o lgb
#son 4 dias a predecir el comportamiento de los prosumers
#predecir con rolling de a 1 dia


#analizar y preparar
# como siempre primero ver los faltantes y los NaN y en cada caso decidir que hacer
# atento con valores expresados para el dia actual o la hora actual / la hora anterior / el dia de mañana , ver de no malinterpretar
# ver la correlacion de precios sea gas y/o electricidad para ver como se comparta con business y no business
# lo mismo con el clima para business y no business
# ver si aplica el supuesto de que los business si operan bajo esos valores (costos de electricidad/gas) y los no business no es tan relevante
# clima historical vs forecast , adaptar columnas que usan distintas proporciones , medir la certeza del forecast
# analizar por fuera como funcionan los paneles y como miden lo que miden ( para mejor entendimiento del df)

#columnas a agregar
#Datetime dias lu-1/ma-2/mi-3/ju-4/vi-5/sa-6/do-7 
#Date time horas y capaz minutos
#horario laboral ejemplo 7 a 17hs (analizarlo) sea en el dataset y por fuera
#horario business , para los que son business ver un pseudo horario de apertura y cierre como afecta y si se puede
#Dia 6hs a 18hs / noche de 18hs a 6hs ver si lo adapto con 4 columnas por las estaciones
#hora de dormir ejemplo 22hs a 6hs
#dias festivos y vacaciones , columnas binarias en ambos
#estaciones del año 4 columnas binarias
#periodogram para ver los lags y sea mensuales/quincenales/diarios/hora
#ver si es relevante agregar alguna columna con la poblacion mensual y usar un ratio (info a buscar afuera)

#sugerencias de features

##Energéticas:

# Ratio production/consumption por prediction_unit_id (histórico)
# Capacidad instalada per capita (installed_capacity / eic_count)
# Eficiencia de paneles (production vs solar_radiation cuando is_consumption=False)
# Volatilidad de precios (rolling std de electricity/gas prices)
# Spread gas-electricidad (diferencia de precios, importante para switching)

##Temporales:

# Lag de consumo/producción del mismo prediction_unit_id (24h, 48h, 168h)
# Media móvil de target por prediction_unit_id
# Cambios día a día (delta vs día anterior)

##Weather engineering:

# Índice de confort térmico (combinando temp + humidity)
# Potencial solar real (direct + diffuse radiation ajustado por cloudcover)
# Diferencia forecast vs historical weather (para medir accuracy del forecast)

##Segmentación:

# Clustering de prediction_units por comportamiento similar
# Ratios county-level (agregaciones por county vs individual)


In [5]:
# hacer scatterplot con 
# train3.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1) el alpha para ver con mas claridad las zonas mas poblada
# podria agregar colores para diferencia temperatura o exposicion a rayos solares o nubes y poder encarar distintos el dataset
# sino funciona probar %matplotlib inline
# train3.hist(bins=50, figsize=(20,15)) ver tmb los histograma de los datos cuando este todo junto# cambiar los bins a valores mas chicos
# plt.show() agregar si hace falta
# corr_matrix = train3.corr()
# corr_matrix["target"].sort_values(ascending=False) ver si aplica pero creo que no , recordad probar en columnas numericas si hay otra relacion
# ejemplo crear columna con raiz/potencia/log y ver como apartan esos valores en correlacion
# o usar scatter_matrix() de pandas

In [6]:
train                 = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/train.csv")
client                = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/client.csv')
electricity_prices    = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv')
forecast_weather      = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv')
gas_prices            = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv')
historical_weather    = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv')
weather_station       = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv')

test                  = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/test.csv')
client_t              = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/client.csv')
electricity_prices_t  = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/electricity_prices.csv')
forecast_weather_t    = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/forecast_weather.csv')
gas_prices_t          = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/gas_prices.csv')
revealed_targets      = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/revealed_targets.csv')


In [7]:
# pruebo pipeline

In [8]:
import unicodedata
from sklearn.neighbors import NearestNeighbors
import gc
from collections import defaultdict
from pathlib import Path

class DataPreprocessingPipeline:
    """Pipeline modular para preprocessing de datos de train y test"""
    
    def __init__(self):
        self.county_id_to_name_map = {
            0: "HARJUMAA", 1: "HIIUMAA", 2: "IDA-VIRUMAA", 3: "JÄRVAMAA",
            4: "JÕGEVAMAA", 5: "LÄÄNE-VIRUMAA", 6: "LÄÄNEMAA", 7: "PÄRNUMAA",
            8: "PÕLVAMAA", 9: "RAPLAMAA", 10: "SAAREMAA", 11: "TARTUMAA",
            12: "UNKNOWN", 13: "VALGAMAA", 14: "VILJANDIMAA", 15: "VÕRUMAA"
        }
        self.county_name_to_id_map = {v: k for k, v in self.county_id_to_name_map.items()}
        
        # Diccionario de sinónimos para normalización
        self.synonyms_to_canonical = {
            "HARJU": "HARJUMAA", "HIIU": "HIIUMAA", "IDA-VIRU": "IDA-VIRUMAA",
            "JARVA": "JÄRVAMAA", "JÄRVA": "JÄRVAMAA", "JOGEVA": "JÕGEVAMAA", 
            "JÕGEVA": "JÕGEVAMAA", "LAANE-VIRU": "LÄÄNE-VIRUMAA", 
            "LÄÄNE-VIRU": "LÄÄNE-VIRUMAA", "LAANE": "LÄÄNEMAA", "LÄÄNE": "LÄÄNEMAA",
            "PARNU": "PÄRNUMAA", "PÄRNU": "PÄRNUMAA", "POLVA": "PÕLVAMAA", 
            "PÕLVA": "PÕLVAMAA", "RAPLA": "RAPLAMAA", "SAARE": "SAAREMAA",
            "TARTU": "TARTUMAA", "VALGA": "VALGAMAA", "VILJANDI": "VILJANDIMAA",
            "VORU": "VÕRUMAA", "VÕRU": "VÕRUMAA",
            # versiones ya canónicas
            **{name: name for name in self.county_id_to_name_map.values()}
        }
    
    def strip_accents(self, s: str) -> str:
        """Eliminar acentos de string"""
        return ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
    
    def normalize_county_name(self, x):
        """Normalizar nombre de county"""
        if pd.isna(x):
            return np.nan
        s = str(x).upper().strip()
        # eliminar sufijos frecuentes
        for suf in [" COUNTY", " MAAKOND"]:
            if s.endswith(suf):
                s = s[: -len(suf)]
        s = s.replace("  ", " ").replace("–", "-")
        s_noacc = self.strip_accents(s)
        s_noacc = s_noacc.replace("  ", " ")
        
        token = s_noacc.split()[0]
        if "-" in s_noacc:
            token = s_noacc
        
        # intentos de match
        if s_noacc in self.synonyms_to_canonical:
            return self.synonyms_to_canonical[s_noacc]
        if token in self.synonyms_to_canonical:
            return self.synonyms_to_canonical[token]
        if s in self.synonyms_to_canonical:
            return self.synonyms_to_canonical[s]
        
        return np.nan
    
    def process_weather_stations(self, weather_station):
        """Procesar estaciones meteorológicas para asignar counties"""
        weather_station = weather_station.copy()
        
        # Normalizar nombres de county
        weather_station['county_name_norm'] = weather_station['county_name'].apply(self.normalize_county_name)
        
        # Si tengo 'county' numérico pero falta nombre, lo inferimos del ID
        mask = weather_station['county_name_norm'].isna() & weather_station['county'].notna()
        weather_station.loc[mask, 'county_name_norm'] = (
            weather_station.loc[mask, 'county'].astype(int).map(self.county_id_to_name_map)
        )
        
        # Completar faltantes por k-NN
        fcols = ['latitude', 'longitude']
        known = weather_station.dropna(subset=['county_name_norm']).copy()
        unknown = weather_station[weather_station['county_name_norm'].isna()].copy()
        
        if not unknown.empty and not known.empty:
            known_rad = np.radians(known[fcols].values)
            unknown_rad = np.radians(unknown[fcols].values)
            
            nbrs = NearestNeighbors(n_neighbors=3, metric='haversine')
            nbrs.fit(known_rad)
            distances, idxs = nbrs.kneighbors(unknown_rad)
            
            nn_names = known['county_name_norm'].values
            filled = []
            for neigh_idx in idxs:
                cands = nn_names[neigh_idx]
                vals, counts = np.unique(cands, return_counts=True)
                filled.append(vals[np.argmax(counts)])
            
            weather_station.loc[unknown.index, 'county_name_norm'] = filled
        
        # Sincronizar columnas finales
        weather_station['county_name'] = weather_station['county_name_norm']
        weather_station['county'] = weather_station['county_name'].map(self.county_name_to_id_map).astype('Int64')
        
        # Cualquier remanente a UNKNOWN (12)
        weather_station.loc[weather_station['county_name'].isna(), 'county'] = 12
        weather_station['county'] = weather_station['county'].astype('Int64')
        
        weather_station.drop(columns=['county_name_norm'], inplace=True)
        
        return weather_station
    
    def optimize_dtypes(self, df, is_train=True):
        """Optimizar tipos de datos para reducir memoria"""
        df = df.copy()
        
        # Tipos comunes
        if 'county' in df.columns:
            df['county'] = df['county'].astype('uint8')
        if 'is_business' in df.columns:
            df['is_business'] = df['is_business'].astype('uint8')
        if 'product_type' in df.columns:
            df['product_type'] = df['product_type'].astype('uint8')
        if 'is_consumption' in df.columns:
            df['is_consumption'] = df['is_consumption'].astype('uint8')
        if 'hour' in df.columns:
            df['hour'] = df['hour'].astype('uint8')
        
        # IDs
        if 'data_block_id' in df.columns:
            df['data_block_id'] = df['data_block_id'].astype('uint16')
        if 'row_id' in df.columns:
            df['row_id'] = df['row_id'].astype('uint32')
        if 'prediction_unit_id' in df.columns:
            df['prediction_unit_id'] = df['prediction_unit_id'].astype('uint8')
        
        # Coordenadas y variables meteorológicas - float32
        float_cols = ['latitude', 'longitude', 'target', 'lowest_price_per_mwh', 
                     'highest_price_per_mwh', 'euros_per_mwh', 'eic_count', 
                     'installed_capacity']
        
        # Variables meteorológicas
        weather_cols = ['ftemperature', 'fdewpoint', 'fcloudcover_high', 'fcloudcover_low', 
                       'fcloudcover_mid', 'fcloudcover_total', 'f10_metre_u_wind_component', 
                       'f10_metre_v_wind_component', 'fdirect_solar_radiation', 
                       'fsurface_solar_radiation_downwards', 'fsnowfall', 'ftotal_precipitation',
                       'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure',
                       'cloudcover_total', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_high',
                       'windspeed_10m', 'winddirection_10m', 'shortwave_radiation', 
                       'direct_solar_radiation', 'diffuse_radiation']
        
        # Aplicar float32 a columnas que existen
        for col in float_cols + weather_cols:
            if col in df.columns:
                df[col] = df[col].astype('float32')
        
        # Weather forecast hour
        if 'weather_forecast_hour' in df.columns:
            df['weather_forecast_hour'] = df['weather_forecast_hour'].astype('uint8')
        
        return df


class TrainPipeline(DataPreprocessingPipeline):
    """Pipeline específico para datos de entrenamiento"""
    
    def __init__(self):
        super().__init__()
        self.temp_dir = Path("temp_chunks")
        self.temp_dir.mkdir(exist_ok=True)
    
    def part1_prepare_base_merges(self, train, gas_prices, electricity_prices, client):
        """Parte 1: Preparar datos base y hacer primeros merges"""
        print("=== PARTE 1: Preparación y merges base ===")
        
        # Preparar copias
        train1 = train.dropna().copy()
        gas_prices1 = gas_prices.copy()
        electricity_prices1 = electricity_prices.copy()
        client1 = client.copy()
        
        # Procesar train
        train1['datetime'] = pd.to_datetime(train1['datetime'])
        train1['hour'] = train1['datetime'].dt.hour
        train1['forecast_date'] = train1['datetime']
        
        # Merge 1: Gas prices
        print("Procesando gas prices...")
        gas_prices1['origin_date'] = pd.to_datetime(gas_prices1['origin_date'])
        gas_prices1['forecast_date'] = pd.to_datetime(gas_prices1['forecast_date'])
        gas_prices1 = gas_prices1.rename(columns={'origin_date': 'gas_origin_date'})
        
        # Expandir gas a nivel horario
        gas_hourly = []
        for _, row in gas_prices1.iterrows():
            for hour in range(24):
                new_row = {
                    'forecast_date': pd.to_datetime(row['forecast_date']) + pd.Timedelta(hours=hour),
                    'lowest_price_per_mwh': row['lowest_price_per_mwh'],
                    'highest_price_per_mwh': row['highest_price_per_mwh']
                }
                gas_hourly.append(new_row)
        
        gas_prices1_hourly = pd.DataFrame(gas_hourly)
        train2 = pd.merge(train1, gas_prices1_hourly, on='forecast_date', how='left')
        
        # Forward fill gas prices
        train2['lowest_price_per_mwh'] = train2['lowest_price_per_mwh'].fillna(method='ffill')
        train2['highest_price_per_mwh'] = train2['highest_price_per_mwh'].fillna(method='ffill')
        
        # Merge 2: Electricity prices
        print("Procesando electricity prices...")
        electricity_prices1['forecast_date'] = pd.to_datetime(electricity_prices1['forecast_date'])
        electricity_prices1 = electricity_prices1.drop(columns=['origin_date'])
        
        elec_columns = ['forecast_date', 'euros_per_mwh']
        train3 = pd.merge(train2, electricity_prices1[elec_columns], on='forecast_date', how='left')
        train3['euros_per_mwh'] = train3['euros_per_mwh'].fillna(method='ffill')
        
        # Merge 3: Client data
        print("Procesando client data...")
        client1['date'] = pd.to_datetime(client1['date']).dt.date
        
        train4 = train3.merge(client1.drop('data_block_id', axis=1),
                             left_on=['product_type', 'county', 'is_business', train3['datetime'].dt.date],
                             right_on=['product_type', 'county', 'is_business', 'date'],
                             how='left')
        
        train4 = train4.drop('date', axis=1)
        train4 = train4.rename(columns={'data_block_id_x': 'data_block_id'})
        
        # Forward fill client data
        print("Completando datos de clientes...")
        train4 = train4.sort_values(['county', 'is_business', 'product_type', 'datetime'])
        train4['eic_count'] = train4.groupby(['county', 'is_business', 'product_type'])['eic_count'].fillna(method='ffill')
        train4['installed_capacity'] = train4.groupby(['county', 'is_business', 'product_type'])['installed_capacity'].fillna(method='ffill')
        
        # Rellenar restantes con promedio por county
        train4['eic_count'] = train4['eic_count'].fillna(train4.groupby('county')['eic_count'].transform('mean'))
        train4['installed_capacity'] = train4['installed_capacity'].fillna(train4.groupby('county')['installed_capacity'].transform('mean'))
        
        # Optimizar tipos
        train4 = self.optimize_dtypes(train4, is_train=True)
        
        print(f"Parte 1 completada. Dataset: {train4.shape[0]:,} filas, {train4.shape[1]} columnas")
        return train4
    
    def part2_prepare_weather_merge(self, train4, forecast_weather, historical_weather, weather_station):
        """Parte 2: Preparar datos meteorológicos y hacer merge"""
        print("=== PARTE 2: Preparación datos meteorológicos ===")
        
        # Procesar weather stations
        weather_station_processed = self.process_weather_stations(weather_station)
        
        # Procesar forecast weather
        print("Procesando forecast weather...")
        forecast_weather1 = forecast_weather.dropna().copy()
        
        # Merge forecast con weather stations
        forecast_weather1 = forecast_weather1.merge(
            weather_station_processed[['latitude', 'longitude', 'county', 'county_name']], 
            on=['latitude', 'longitude'], 
            how='left'
        )
        
        # Completar counties faltantes con k-NN
        missing_county = forecast_weather1['county'].isna()
        if missing_county.sum() > 0:
            print(f"Completando {missing_county.sum()} counties faltantes con k-NN...")
            known = weather_station_processed[['latitude', 'longitude', 'county', 'county_name']].copy()
            missing_coords = forecast_weather1.loc[missing_county, ['latitude', 'longitude']].copy()
            
            nbrs = NearestNeighbors(n_neighbors=1, metric='euclidean')
            nbrs.fit(known[['latitude', 'longitude']].values)
            distances, indices = nbrs.kneighbors(missing_coords[['latitude', 'longitude']].values)
            
            nearest_counties = known.iloc[indices.flatten()]
            forecast_weather1.loc[missing_county, 'county'] = nearest_counties['county'].values
            forecast_weather1.loc[missing_county, 'county_name'] = nearest_counties['county_name'].values
        
        # Transformar forecast weather
        forecast_weather2 = forecast_weather1.copy()
        forecast_weather2 = forecast_weather2.drop(['county_name', 'origin_datetime'], axis=1)
        forecast_weather2['forecast_datetime'] = pd.to_datetime(forecast_weather2['forecast_datetime'])
        
        # Renombrar con prefijo 'f'
        rename_dict = {}
        for col in forecast_weather2.columns:
            if col not in ['county', 'hours_ahead', 'latitude', 'longitude', 'data_block_id', 
                          'forecast_datetime']:
                rename_dict[col] = 'f' + col
        
        forecast_weather2 = forecast_weather2.rename(columns=rename_dict)
        forecast_weather2 = forecast_weather2.rename(columns={"hours_ahead": "weather_forecast_hour"})
        
        # Merge con historical weather
        print("Mergeando forecast con historical weather...")
        historical_weather1 = historical_weather.copy()
        historical_weather1['datetime'] = pd.to_datetime(historical_weather1['datetime'])
        
        forecast_weather2 = forecast_weather2.rename(columns={'data_block_id': 'f_data_block_id'})
        
        merged_weather = pd.merge(
            forecast_weather2,
            historical_weather1,
            left_on=['latitude', 'longitude', 'forecast_datetime'],
            right_on=['latitude', 'longitude', 'datetime'],
            how='inner'
        )
        
        # Limpiar y optimizar
        merged_weather = merged_weather.drop(['f_data_block_id', 'forecast_datetime'], axis=1)
        merged_weather = self.optimize_dtypes(merged_weather, is_train=True)
        
        print(f"Weather data preparado: {merged_weather.shape[0]:,} filas, {merged_weather.shape[1]} columnas")
        return train4, merged_weather
    
    def part3_final_merge_and_cleanup(self, train4, merged_weather, chunk_size=1_000_000):
        """Parte 3: Merge final y limpieza"""
        print("=== PARTE 3: Merge final y limpieza ===")
        
        # Definir períodos para procesar por chunks
        periods = [
            ('2021-09-01', '2021-10-01'), ('2021-10-01', '2021-11-01'), ('2021-11-01', '2021-12-01'), 
            ('2021-12-01', '2022-01-01'), ('2022-01-01', '2022-02-01'), ('2022-02-01', '2022-03-01'), 
            ('2022-03-01', '2022-04-01'), ('2022-04-01', '2022-05-01'), ('2022-05-01', '2022-06-01'), 
            ('2022-06-01', '2022-07-01'), ('2022-07-01', '2022-08-01'), ('2022-08-01', '2022-09-01'),
            ('2022-09-01', '2022-10-01'), ('2022-10-01', '2022-11-01'), ('2022-11-01', '2022-12-01'), 
            ('2022-12-01', '2023-01-01'), ('2023-01-01', '2023-02-01'), ('2023-02-01', '2023-03-01'), 
            ('2023-03-01', '2023-04-01'), ('2023-04-01', '2023-05-01'), ('2023-05-01', '2023-05-31')
        ]
        
        # Procesar por chunks temporales
        chunk_files = []
        for i, (start_date, end_date) in enumerate(periods):
            print(f"Procesando período {i+1}/{len(periods)}: {start_date} a {end_date}")
            
            # Filtrar por período
            mask_train = (train4['datetime'] >= start_date) & (train4['datetime'] < end_date)
            train4_chunk = train4[mask_train].copy()
            
            mask_weather = (merged_weather['datetime'] >= start_date) & (merged_weather['datetime'] < end_date)
            weather_chunk = merged_weather[mask_weather].copy()
            
            if len(train4_chunk) > 0 and len(weather_chunk) > 0:
                # Merge chunk
                chunk_result = pd.merge(train4_chunk, weather_chunk, 
                               on=['county', 'datetime'], 
                               how='left')
                
                # Guardar chunk temporal
                chunk_file = self.temp_dir / f'train5_chunk_{i+1}.parquet'
                chunk_result.to_parquet(chunk_file, index=False)
                chunk_files.append(chunk_file)
                
                print(f"Chunk {i+1} guardado: {chunk_result.shape}")
                
                # Liberar memoria
                del train4_chunk, weather_chunk, chunk_result
                gc.collect()
        
        # Concatenar chunks
        print("Concatenando chunks...")
        final_chunks = []
        for chunk_file in chunk_files:
            chunk_data = pd.read_parquet(chunk_file)
            final_chunks.append(chunk_data)
        
        train5 = pd.concat(final_chunks, ignore_index=True)
        
        # Limpieza final
        print("Aplicando limpieza final...")
        train5 = self._cleanup_merged_data(train5, chunk_size)
        
        # Limpiar archivos temporales
        for chunk_file in chunk_files:
            chunk_file.unlink()
        
        return train5
    
    def _cleanup_merged_data(self, train5, chunk_size=1_000_000):
        """Limpiar datos después del merge"""
        # Identificar columnas a eliminar y renombrar
        sample = train5.head(1000)
        cols_to_drop = [col for col in sample.columns if col.endswith('_y')]
        rename_dict = {col: col[:-2] for col in sample.columns if col.endswith('_x')}
        
        print(f"Eliminando {len(cols_to_drop)} columnas, renombrando {len(rename_dict)}")
        
        # Procesar por chunks
        chunks_processed = []
        total_rows = len(train5)
        n_chunks = (total_rows // chunk_size) + 1
        
        for i in range(n_chunks):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, total_rows)
            
            print(f"Procesando chunk limpieza {i+1}/{n_chunks}: filas {start_idx:,} a {end_idx:,}")
            
            chunk = train5.iloc[start_idx:end_idx].copy()
            
            # Filtrar county != 12
            chunk = chunk[chunk['county'] != 12]
            
            if len(chunk) > 0:
                # Eliminar columnas _y y renombrar _x
                chunk = chunk.drop(columns=cols_to_drop)
                chunk.rename(columns=rename_dict, inplace=True)
                chunks_processed.append(chunk)
            
            del chunk
            gc.collect()
        
        # Concatenar final
        train5_clean = pd.concat(chunks_processed, ignore_index=True)
        del chunks_processed
        gc.collect()
        
        # Eliminar datos de clima faltantes
        weather_key_cols = ['latitude', 'longitude', 'ftemperature', 'temperature']
        initial_rows = len(train5_clean)
        train5_clean = train5_clean.dropna(subset=weather_key_cols)
        print(f"Filas eliminadas por clima faltante: {initial_rows - len(train5_clean):,}")
        
        # Reordenar columnas
        train5_clean = self._reorder_columns(train5_clean)
        
        return train5_clean
    
    def _reorder_columns(self, df):
        """Reordenar columnas en el orden especificado"""
        main_cols = [
            'row_id', 'target', 'datetime', 'forecast_date', 'hour', 'data_block_id', 
            'prediction_unit_id', 'is_business', 'product_type', 'county', 'latitude', 
            'longitude', 'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh', 
            'eic_count', 'installed_capacity'
        ]
        
        weather_pairs = [
            'ftemperature', 'temperature', 'fdewpoint', 'dewpoint',
            'fcloudcover_high', 'cloudcover_high', 'fcloudcover_low', 'cloudcover_low', 
            'fcloudcover_mid', 'cloudcover_mid', 'fcloudcover_total', 'cloudcover_total',
            'f10_metre_u_wind_component', 'windspeed_10m', 'f10_metre_v_wind_component', 
            'winddirection_10m', 'fdirect_solar_radiation', 'direct_solar_radiation',
            'fsurface_solar_radiation_downwards', 'shortwave_radiation', 'diffuse_radiation',
            'fsnowfall', 'snowfall', 'ftotal_precipitation', 'rain', 'surface_pressure', 
            'weather_forecast_hour'
        ]
        
        final_order = main_cols.copy()
        for col in weather_pairs:
            if col in df.columns:
                final_order.append(col)
        
        remaining_cols = [col for col in df.columns if col not in final_order]
        final_order.extend(remaining_cols)
        
        return df[final_order]
    
    def run_full_pipeline(self, train, gas_prices, electricity_prices, client, 
                         forecast_weather, historical_weather, weather_station):
        """Ejecutar pipeline completo de entrenamiento"""
        print("Iniciando pipeline completo de entrenamiento...")
        
        # Parte 1: Merges base
        train4 = self.part1_prepare_base_merges(train, gas_prices, electricity_prices, client)
        
        # Parte 2: Preparar datos meteorológicos
        train4, merged_weather = self.part2_prepare_weather_merge(
            train4, forecast_weather, historical_weather, weather_station
        )
        
        # Parte 3: Merge final y limpieza
        train5 = self.part3_final_merge_and_cleanup(train4, merged_weather)
        
        print(f"Pipeline completado! Dataset final: {train5.shape[0]:,} filas, {train5.shape[1]} columnas")
        return train5


class TestPipeline(DataPreprocessingPipeline):
    """Pipeline simplificado para datos de test"""
    
    def run_test_pipeline(self, test, gas_prices_t, electricity_prices_t, client_t, 
                         forecast_weather_t, weather_station):
        """Pipeline completo para test set (más simple, sin historical weather)"""
        print("=== PIPELINE TEST ===")
        
        # Preparar datos base
        test1 = test.copy()
        test1['datetime'] = pd.to_datetime(test1['prediction_datetime'])
        test1['hour'] = test1['datetime'].dt.hour
        test1['forecast_date'] = test1['datetime']
        
        # Merge gas prices
        print("Procesando gas prices...")
        gas_prices1 = gas_prices_t.copy()
        gas_prices1['origin_date'] = pd.to_datetime(gas_prices1['origin_date'])
        gas_prices1['forecast_date'] = pd.to_datetime(gas_prices1['forecast_date'])
        
        gas_hourly = []
        for _, row in gas_prices1.iterrows():
            for hour in range(24):
                new_row = {
                    'forecast_date': pd.to_datetime(row['forecast_date']) + pd.Timedelta(hours=hour),
                    'lowest_price_per_mwh': row['lowest_price_per_mwh'],
                    'highest_price_per_mwh': row['highest_price_per_mwh']
                }
                gas_hourly.append(new_row)
        
        gas_prices_hourly = pd.DataFrame(gas_hourly)
        test2 = pd.merge(test1, gas_prices_hourly, on='forecast_date', how='left')
        test2['lowest_price_per_mwh'] = test2['lowest_price_per_mwh'].fillna(method='ffill')
        test2['highest_price_per_mwh'] = test2['highest_price_per_mwh'].fillna(method='ffill')
        
        # Merge electricity prices
        print("Procesando electricity prices...")
        electricity_prices1 = electricity_prices_t.copy()
        electricity_prices1['forecast_date'] = pd.to_datetime(electricity_prices1['forecast_date'])
        electricity_prices1 = electricity_prices1.drop(columns=['origin_date'])
        
        test3 = pd.merge(test2, electricity_prices1[['forecast_date', 'euros_per_mwh']], 
                        on='forecast_date', how='left')
        test3['euros_per_mwh'] = test3['euros_per_mwh'].fillna(method='ffill')
        
        # Merge client data
        print("Procesando client data...")
        client1 = client_t.copy()
        client1['date'] = pd.to_datetime(client1['date']).dt.date
        
        test4 = test3.merge(client1.drop('data_block_id', axis=1),
                           left_on=['product_type', 'county', 'is_business', test3['datetime'].dt.date],
                           right_on=['product_type', 'county', 'is_business', 'date'],
                           how='left')
        
        test4 = test4.drop('date', axis=1)
        test4 = test4.rename(columns={'data_block_id_x': 'data_block_id'})
        
        # Forward fill client data
        test4 = test4.sort_values(['county', 'is_business', 'product_type', 'datetime'])
        test4['eic_count'] = test4.groupby(['county', 'is_business', 'product_type'])['eic_count'].fillna(method='ffill')
        test4['installed_capacity'] = test4.groupby(['county', 'is_business', 'product_type'])['installed_capacity'].fillna(method='ffill')
        
        # Rellenar restantes con promedio por county
        test4['eic_count'] = test4['eic_count'].fillna(test4.groupby('county')['eic_count'].transform('mean'))
        test4['installed_capacity'] = test4['installed_capacity'].fillna(test4.groupby('county')['installed_capacity'].transform('mean'))
        
        # Procesar forecast weather
        print("Procesando forecast weather...")
        weather_station_processed = self.process_weather_stations(weather_station)
        
        forecast_weather1 = forecast_weather_t.copy()
        
        # Merge forecast con weather stations
        forecast_weather1 = forecast_weather1.merge(
            weather_station_processed[['latitude', 'longitude', 'county', 'county_name']], 
            on=['latitude', 'longitude'], 
            how='left'
        )
        
        # Completar counties faltantes con k-NN
        missing_county = forecast_weather1['county'].isna()
        if missing_county.sum() > 0:
            print(f"Completando {missing_county.sum()} counties faltantes con k-NN...")
            known = weather_station_processed[['latitude', 'longitude', 'county', 'county_name']].copy()
            missing_coords = forecast_weather1.loc[missing_county, ['latitude', 'longitude']].copy()
            
            nbrs = NearestNeighbors(n_neighbors=1, metric='euclidean')
            nbrs.fit(known[['latitude', 'longitude']].values)
            distances, indices = nbrs.kneighbors(missing_coords[['latitude', 'longitude']].values)
            
            nearest_counties = known.iloc[indices.flatten()]
            forecast_weather1.loc[missing_county, 'county'] = nearest_counties['county'].values
            forecast_weather1.loc[missing_county, 'county_name'] = nearest_counties['county_name'].values
        
        # Transformar forecast weather
        forecast_weather2 = forecast_weather1.copy()
        forecast_weather2 = forecast_weather2.drop(['county_name', 'origin_datetime'], axis=1)
        forecast_weather2['forecast_datetime'] = pd.to_datetime(forecast_weather2['forecast_datetime'])
        
        # Renombrar con prefijo 'f'
        rename_dict = {}
        for col in forecast_weather2.columns:
            if col not in ['county', 'hours_ahead', 'latitude', 'longitude', 'data_block_id', 
                          'forecast_datetime']:
                rename_dict[col] = 'f' + col
        
        forecast_weather2 = forecast_weather2.rename(columns=rename_dict)
        forecast_weather2 = forecast_weather2.rename(columns={"hours_ahead": "weather_forecast_hour"})
        
        # Para test, no tenemos historical weather, así que creamos columnas dummy o usamos solo forecast
        # Opción 1: Solo usar forecast weather (renombrar las f columns)
        # Opción 2: Crear columnas históricas como NaN y llenar después
        
        # Vamos con opción 1: usar forecast como histórico también (aproximación)
        historical_cols_mapping = {
            'ftemperature': 'temperature',
            'fdewpoint': 'dewpoint', 
            'fcloudcover_high': 'cloudcover_high',
            'fcloudcover_low': 'cloudcover_low',
            'fcloudcover_mid': 'cloudcover_mid', 
            'fcloudcover_total': 'cloudcover_total',
            'fdirect_solar_radiation': 'direct_solar_radiation',
            'fsurface_solar_radiation_downwards': 'shortwave_radiation',
            'fsnowfall': 'snowfall',
            'ftotal_precipitation': 'rain'
        }
        
        # Crear versiones históricas basadas en forecast
        for fcol, hcol in historical_cols_mapping.items():
            if fcol in forecast_weather2.columns:
                forecast_weather2[hcol] = forecast_weather2[fcol]
        
        # Agregar columnas que solo existen en historical
        forecast_weather2['surface_pressure'] = 1013.25  # valor típico
        forecast_weather2['windspeed_10m'] = np.sqrt(
            forecast_weather2['f10_metre_u_wind_component']**2 + 
            forecast_weather2['f10_metre_v_wind_component']**2
        ) if 'f10_metre_u_wind_component' in forecast_weather2.columns else 0
        
        forecast_weather2['winddirection_10m'] = np.arctan2(
            forecast_weather2['f10_metre_v_wind_component'], 
            forecast_weather2['f10_metre_u_wind_component']
        ) * 180 / np.pi if 'f10_metre_v_wind_component' in forecast_weather2.columns else 0
        
        forecast_weather2['diffuse_radiation'] = forecast_weather2.get('fsurface_solar_radiation_downwards', 0) - forecast_weather2.get('fdirect_solar_radiation', 0)
        forecast_weather2['diffuse_radiation'] = forecast_weather2['diffuse_radiation'].clip(lower=0)
        
        # Limpiar y optimizar
        forecast_weather2 = forecast_weather2.drop(columns=['data_block_id'], errors='ignore')
        forecast_weather2 = self.optimize_dtypes(forecast_weather2, is_train=False)
        
        # Merge final con weather
        print("Merge final con datos meteorológicos...")
        test5 = pd.merge(test4, forecast_weather2, 
                        left_on=['datetime', 'county'], 
                        right_on=['forecast_datetime', 'county'], 
                        how='left')



        # Limpiar columnas duplicadas y renombrar
        cols_to_drop = [col for col in test5.columns if col.endswith('_y')]
        rename_dict = {col: col[:-2] for col in test5.columns if col.endswith('_x')}
        
        test5 = test5.drop(columns=cols_to_drop + ['forecast_datetime'], errors='ignore')
        test5.rename(columns=rename_dict, inplace=True)
        
        # Optimizar tipos finales
        test5 = self.optimize_dtypes(test5, is_train=False)
        
        # Reordenar columnas igual que train
        test5 = self._reorder_columns_test(test5)
        
        print(f"Pipeline test completado! Dataset: {test5.shape[0]:,} filas, {test5.shape[1]} columnas")
        return test5
    
    def _reorder_columns_test(self, df):
        """Reordenar columnas para test (mismo orden que train)"""
        main_cols = [
            'row_id', 'datetime', 'forecast_date', 'hour', 'data_block_id', 
            'prediction_unit_id', 'is_business', 'product_type', 'county', 'latitude', 
            'longitude', 'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh', 
            'eic_count', 'installed_capacity'
        ]
        
        weather_pairs = [
            'ftemperature', 'temperature', 'fdewpoint', 'dewpoint',
            'fcloudcover_high', 'cloudcover_high', 'fcloudcover_low', 'cloudcover_low', 
            'fcloudcover_mid', 'cloudcover_mid', 'fcloudcover_total', 'cloudcover_total',
            'f10_metre_u_wind_component', 'windspeed_10m', 'f10_metre_v_wind_component', 
            'winddirection_10m', 'fdirect_solar_radiation', 'direct_solar_radiation',
            'fsurface_solar_radiation_downwards', 'shortwave_radiation', 'diffuse_radiation',
            'fsnowfall', 'snowfall', 'ftotal_precipitation', 'rain', 'surface_pressure', 
            'weather_forecast_hour'
        ]
        
        final_order = []
        # Solo agregar columnas que existen
        for col in main_cols:
            if col in df.columns:
                final_order.append(col)
        
        for col in weather_pairs:
            if col in df.columns:
                final_order.append(col)
        
        remaining_cols = [col for col in df.columns if col not in final_order]
        final_order.extend(remaining_cols)
        
        return df[final_order]


# Funciones de utilidad para usar el pipeline
def run_train_pipeline(train, gas_prices, electricity_prices, client, 
                      forecast_weather, historical_weather, weather_station):
    """
    Función principal para ejecutar pipeline de entrenamiento
    
    Returns:
        pd.DataFrame: Dataset procesado train5
    """
    pipeline = TrainPipeline()
    return pipeline.run_full_pipeline(
        train, gas_prices, electricity_prices, client, 
        forecast_weather, historical_weather, weather_station
    )

def run_test_pipeline(test, gas_prices_t, electricity_prices_t, client_t, 
                     forecast_weather_t, weather_station):
    """
    Función principal para ejecutar pipeline de test
    
    Returns:
        pd.DataFrame: Dataset procesado test5
    """
    pipeline = TestPipeline()
    return pipeline.run_test_pipeline(
        test, gas_prices_t, electricity_prices_t, client_t, 
        forecast_weather_t, weather_station
    )

# Ejemplo de uso:
"""
# Para train:
train5 = run_train_pipeline(
    train, gas_prices, electricity_prices, client, 
    forecast_weather, historical_weather, weather_station
)

# Para test:
test5 = run_test_pipeline(
    test, gas_prices_t, electricity_prices_t, client_t, 
    forecast_weather_t, weather_station
)
"""

'\n# Para train:\ntrain5 = run_train_pipeline(\n    train, gas_prices, electricity_prices, client, \n    forecast_weather, historical_weather, weather_station\n)\n\n# Para test:\ntest5 = run_test_pipeline(\n    test, gas_prices_t, electricity_prices_t, client_t, \n    forecast_weather_t, weather_station\n)\n'

In [9]:
train5 = run_train_pipeline(
    train, gas_prices, electricity_prices, client, 
    forecast_weather, historical_weather, weather_station
)

Iniciando pipeline completo de entrenamiento...
=== PARTE 1: Preparación y merges base ===
Procesando gas prices...
Procesando electricity prices...
Procesando client data...
Completando datos de clientes...
Parte 1 completada. Dataset: 2,017,824 filas, 16 columnas
=== PARTE 2: Preparación datos meteorológicos ===
Procesando forecast weather...
Completando 2140318 counties faltantes con k-NN...
Mergeando forecast con historical weather...
Weather data preparado: 3,418,242 filas, 32 columnas
=== PARTE 3: Merge final y limpieza ===
Procesando período 1/21: 2021-09-01 a 2021-10-01
Chunk 1 guardado: (1423080, 46)
Procesando período 2/21: 2021-10-01 a 2021-11-01
Chunk 2 guardado: (1532066, 46)
Procesando período 3/21: 2021-11-01 a 2021-12-01
Chunk 3 guardado: (1484640, 46)
Procesando período 4/21: 2021-12-01 a 2022-01-01
Chunk 4 guardado: (1543056, 46)
Procesando período 5/21: 2022-01-01 a 2022-02-01
Chunk 5 guardado: (1590672, 46)
Procesando período 6/21: 2022-02-01 a 2022-03-01
Chunk 6 gu

In [10]:
train5.tail()

,row_id,target,datetime,forecast_date,hour,data_block_id,prediction_unit_id,is_business,product_type,county,latitude,longitude,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity,ftemperature,temperature,fdewpoint,dewpoint,fcloudcover_high,cloudcover_high,fcloudcover_low,cloudcover_low,fcloudcover_mid,cloudcover_mid,fcloudcover_total,cloudcover_total,f10_metre_u_wind_component,windspeed_10m,f10_metre_v_wind_component,winddirection_10m,fdirect_solar_radiation,direct_solar_radiation,fsurface_solar_radiation_downwards,shortwave_radiation,diffuse_radiation,fsnowfall,snowfall,ftotal_precipitation,rain,surface_pressure,weather_forecast_hour,is_consumption
32965017,2013541,444.356995,2023-05-30 10:00:00,2023-05-30 10:00:00,10,636,60,1,3,15,57.599998,28.200001,29.0,34.0,82.370003,55.0,2188.199951,12.136377,13.8,2.330225,3.9,0.0,0.0,0.000000,7.0,0.966461,71.0,0.966461,49.0,-1.009770,2.055556,-0.927298,346.0,1.858164,155.0,191.511948,410.0,255.0,0.0,0.0,0.000000,0.1,1012.799988,8.0,1
32965018,2013541,444.356995,2023-05-30 10:00:00,2023-05-30 10:00:00,10,636,60,1,3,15,57.900002,26.700001,29.0,34.0,82.370003,55.0,2188.199951,14.534815,13.9,2.355859,3.3,0.0,0.0,0.270050,15.0,0.544281,73.0,0.652069,57.0,2.205562,2.916667,-1.008352,297.0,832.791504,249.0,685.467529,490.0,241.0,0.0,0.0,0.000056,0.1,1005.200012,8.0,1
32965019,2013541,444.356995,2023-05-30 10:00:00,2023-05-30 10:00:00,10,636,60,1,3,15,57.900002,27.200001,29.0,34.0,82.370003,55.0,2188.199951,13.503565,13.9,3.533350,2.9,0.0,0.0,0.024902,18.0,0.181091,66.0,0.184845,56.0,0.538569,3.194444,-1.362844,309.0,486.835938,257.0,500.649719,498.0,241.0,0.0,0.0,0.000000,0.0,1007.200012,8.0,1
32965020,2013541,444.356995,2023-05-30 10:00:00,2023-05-30 10:00:00,10,636,60,1,3,15,57.900002,27.700001,29.0,34.0,82.370003,55.0,2188.199951,13.150782,13.6,2.600976,3.2,0.0,0.0,0.000153,18.0,0.866364,66.0,0.866455,56.0,-0.223149,3.527778,-1.600393,317.0,26.035942,240.0,268.223053,482.0,242.0,0.0,0.0,0.000000,0.0,1011.099976,8.0,1
32965021,2013541,444.356995,2023-05-30 10:00:00,2023-05-30 10:00:00,10,636,60,1,3,15,57.900002,28.200001,29.0,34.0,82.370003,55.0,2188.199951,12.648584,13.6,1.409814,3.6,0.0,0.0,0.000000,9.0,0.997070,78.0,0.997070,55.0,-0.198003,3.194444,-2.190237,319.0,63.440388,186.0,304.685272,446.0,260.0,0.0,0.0,0.000000,0.0,1013.000000,8.0,1


In [11]:
# Features temporales
train5['day_of_week'] = train5['datetime'].dt.dayofweek
train5['month'] = train5['datetime'].dt.month
# Crear is_daylight (1 si hay radiación solar, 0 si es noche)
train5['is_daylight'] = (train5['direct_solar_radiation'] > 0).astype(int)
# Crear weekend (1 para sábado/domingo, 0 para lunes-viernes)
train5['weekend'] = (train5['day_of_week'] >= 5).astype(int)

In [12]:
#Mae puesto n1 kaggle 52.3090
#1er primer modelo xgb sobre train5 haciendo split 80/20 de train y test 
#Mae = 107.9896
#sin lag sin feature, sin nada , solo haciendo el merge


#agrego day of week y month para todos los siguientes

#2do modelo lgb 10% y sobre esto split 80/20 de train y test 
#Mae = 89.3

#3er modelo lgb 10% y sobre esto split 80/20 de train y test 
#Mae = 103.1249
#climate_vars = ['fsurface_solar_radiation_downwards', 'shortwave_radiation', 
#                'temperature', 'direct_solar_radiation']
# agrego lags de 1h/4h/12h en las variables de arriba
# empeoro porque agregue los lags al azar no hice un analisis para agregarlo en los marcos de tiempo relevante
# estas variables de lags las quito

#agrego is_daylight binaria

#4to modelo lgb 14.2% y sobre esto split 80/20 de train y test  
#de base solo agrego la variable is_daylight , pero ahora uso info de los ultimos 3 meses 14.2% aprox
# se que no deberia ser comparable al usar de base distinto , pero parte de aca ahora
#Mae = 86.6747

#agrego weekend binaria

#5to modelo lgb igual que el anterior , solo agregue binaria Weekend y no vi cambios
#Mae = 86.6747

#para todos los demas modelos ya tienen incluido
#day of week
#month
#is_daylight
#weekend

#6to modelo lgb , agregue variable binaria is_active_hours , horario de 8 a 17 hs binaria
#Mae = 90.4776 , empeoro el modelo la saco por ahora a is_active_hours

#7mo modelo lgb, agregue variable binaria is_sleeping_hours 22h-6h
#Mae = 98.3152 empeora el modelo , asi que  la saco a is_sleeping_hours

#empiezo a probar con variables
##Energéticas:

# Ratio production/consumption por prediction_unit_id (histórico)
# Capacidad instalada per capita (installed_capacity / eic_count)
# Eficiencia de paneles (production vs solar_radiation cuando is_consumption=False)
# Volatilidad de precios (rolling std de electricity/gas prices)
# Spread gas-electricidad (diferencia de precios, importante para switching)

#8vo modelo lgb agregue 3 variables energeticas capacity_per_obs, production_obs_ratio, consumption_obs_ratio
#Mae = 76.2703 mejor bastante el modelo

#9no modelo lgb mantengo las 3 anteriores y prueba una nueva Ratio production/cn

In [13]:
# 2do modelo lgb 10% y sobre esto split 80/20 de train y test 
#Mae = 89.3

# # Sample 10% manteniendo orden temporal
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])
# sample_df = train5.sample(frac=0.1, random_state=42).sort_values(['prediction_unit_id', 'datetime'])

# print(f"Dataset original: {len(train5):,}")
# print(f"Sample 10%: {len(sample_df):,}")

# # Convertir datetime
# sample_df['datetime'] = pd.to_datetime(sample_df['datetime'])
# sample_df['day_of_week'] = sample_df['datetime'].dt.dayofweek
# sample_df['month'] = sample_df['datetime'].dt.month

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# # LightGBM (más eficiente en memoria)
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE: {mae:.4f}")
# print(f"Features usadas: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# Dataset original: 32,963,024
# Sample 10%: 3,296,302
# MAE: 89.3026
# Features usadas: 43
# Train: 2,637,041 | Test: 659,261

In [14]:
# # Feature importance
# importance = model.feature_importances_
# feature_names = X_train.columns

# # Crear DataFrame y ordenar
# importance_df = pd.DataFrame({
#     'feature': feature_names,
#     'importance': importance
# }).sort_values('importance', ascending=False)

# # Ver top 15
# print("Top 15 features más importantes:")
# print(importance_df.head(15))

# # Plot opcional (si quieres visualizar)
# plt.figure(figsize=(10, 8))
# plt.barh(importance_df.head(15)['feature'], importance_df.head(15)['importance'])
# plt.title('Top 15 Feature Importance')
# plt.tight_layout()
# plt.show()

In [15]:
# importance_df['importance_pct'] = (importance_df['importance'] / importance_df['importance'].sum()) * 100
# print(importance_df.head(15)[['feature', 'importance', 'importance_pct']])

#                               feature  importance  importance_pct
# 40                      is_consumption         469       15.633333
# 12                  installed_capacity         436       14.533333
# 11                           eic_count         292        9.733333
# 2                   prediction_unit_id         228        7.600000
# 31  fsurface_solar_radiation_downwards         227        7.566667
# 0                                 hour         180        6.000000
# 1                        data_block_id         164        5.466667
# 41                         day_of_week         161        5.366667
# 32                 shortwave_radiation         111        3.700000
# 3                          is_business         105        3.500000
# 10                       euros_per_mwh          83        2.766667
# 14                         temperature          73        2.433333
# 42                               month          72        2.400000
# 30              direct_solar_radiation          60        2.000000
# 29             fdirect_solar_radiation          48        1.600000

In [16]:
#3er modelo lgb 10% y sobre esto split 80/20 de train y test 
# Mae = 103.1249
#climate_vars = ['fsurface_solar_radiation_downwards', 'shortwave_radiation', 
#                'temperature', 'direct_solar_radiation']
# agrego lags de 1h/4h/12h en las variables de arriba
# empeoro porque agregue los lags al azar no hice un analisis para agregarlo en los marcos de tiempo relevante
# Sample 10%




# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])
# sample_df = train5.sample(frac=0.1, random_state=42).sort_values(['prediction_unit_id', 'datetime'])

# print(f"Sample 10%: {len(sample_df):,}")

# # Convertir datetime
# sample_df['datetime'] = pd.to_datetime(sample_df['datetime'])
# sample_df['day_of_week'] = sample_df['datetime'].dt.dayofweek
# sample_df['month'] = sample_df['datetime'].dt.month

# # Variables climáticas importantes para lags
# climate_vars = ['fsurface_solar_radiation_downwards', 'shortwave_radiation', 
#                 'temperature', 'direct_solar_radiation']

# # Crear lags por prediction_unit_id
# print("🔄 Creando lags climáticos...")
# lag_dfs = []

# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id].copy()
    
#     # Lags de 1h, 4h, 12h para cada variable climática
#     for var in climate_vars:
#         if var in unit_data.columns:
#             unit_data[f'{var}_lag1h'] = unit_data[var].shift(1)
#             unit_data[f'{var}_lag4h'] = unit_data[var].shift(4)
#             unit_data[f'{var}_lag12h'] = unit_data[var].shift(12)
    
#     lag_dfs.append(unit_data)

# sample_with_lags = pd.concat(lag_dfs, ignore_index=True)
# sample_with_lags = sample_with_lags.sort_values(['prediction_unit_id', 'datetime'])

# print(f"Datos con lags: {len(sample_with_lags):,}")

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_with_lags.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal
# split_idx = int(len(sample_with_lags) * 0.8)
# train_data = sample_with_lags.iloc[:split_idx]
# test_data = sample_with_lags.iloc[split_idx:]

# # Dropear NaN (por los lags)
# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train final: {len(X_train):,}")
# print(f"Test final: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE con lags: {mae:.4f}")
# print(f"Mejora vs 89.3: {89.3 - mae:.1f}")

In [17]:
#

In [18]:
#4to modelo lgb de base solo agrego la variable is_daylight , pero ahora uso info de los ultimos 3 meses por memoria 
#Mae = 86.6747


# Preparar datos y tomar últimos 3 meses
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])
# train5['datetime'] = pd.to_datetime(train5['datetime'])

# # Últimos 3 meses
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# print(f"Dataset original: {len(train5):,}")
# print(f"Últimos 3 meses: {len(sample_df):,}")
# print(f"Porcentaje: {len(sample_df)/len(train5)*100:.1f}%")

# # Features temporales
# sample_df['day_of_week'] = sample_df['datetime'].dt.dayofweek
# sample_df['month'] = sample_df['datetime'].dt.month

# # Crear is_daylight (1 si hay radiación solar, 0 si es noche)
# sample_df['is_daylight'] = (sample_df['direct_solar_radiation'] > 0).astype(int)

# print(f"Daylight distribution:")
# print(sample_df['is_daylight'].value_counts())

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE con is_daylight (3 meses): {mae:.4f}")
# print(f"Baseline original era: 89.3")

# # #Dataset original: 32,963,024
# # Últimos 3 meses: 4,674,024
# # Porcentaje: 14.2%
# # Daylight distribution:
# # is_daylight
# # 1    2592660
# # 0    2081364
# # Name: count, dtype: int64
# # Features totales: 44
# # Train: 3,739,219 | Test: 934,805
# # MAE con is_daylight (3 meses): 86.6747
# # Baseline original era: 89.3

In [19]:
#5to modelo lgb igual que el anterior , solo agregue binaria Weekend y no vi cambios
#Mae = 86.6747
# Preparar datos y tomar últimos 3 meses
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])
# train5['datetime'] = pd.to_datetime(train5['datetime'])

# # Últimos 3 meses
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# print(f"Dataset original: {len(train5):,}")
# print(f"Últimos 3 meses: {len(sample_df):,}")
# print(f"Porcentaje: {len(sample_df)/len(train5)*100:.1f}%")

# # Features temporales
# sample_df['day_of_week'] = sample_df['datetime'].dt.dayofweek
# sample_df['month'] = sample_df['datetime'].dt.month

# # Crear is_daylight (1 si hay radiación solar, 0 si es noche)
# sample_df['is_daylight'] = (sample_df['direct_solar_radiation'] > 0).astype(int)

# # Crear weekend (1 para sábado/domingo, 0 para lunes-viernes)
# sample_df['weekend'] = (sample_df['day_of_week'] >= 5).astype(int)

# print(f"Daylight distribution:")
# print(sample_df['is_daylight'].value_counts())
# print(f"Weekend distribution:")
# print(sample_df['weekend'].value_counts())

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE con is_daylight + weekend (3 meses): {mae:.4f}")
# print(f"MAE anterior era: 86.67")

# # Dataset original: 32,963,024
# # Últimos 3 meses: 4,674,024
# # Porcentaje: 14.2%
# # Daylight distribution:
# # is_daylight
# # 1    2592660
# # 0    2081364
# # Name: count, dtype: int64
# # Weekend distribution:
# # weekend
# # 0    3341592
# # 1    1332432
# # Name: count, dtype: int64
# # Features totales: 45
# # Train: 3,739,219 | Test: 934,805
# # MAE con is_daylight + weekend (3 meses): 86.6747
# # MAE anterior era: 86.67

In [20]:
#6to modelo lgb , agregue variable binaria is_active_hours , horario de 8 a 17 hs binaria
#Mae = 90.4776 , empeoro el modelo

# # Usar train5 con las features ya agregadas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # Agregar is_active_hours (horario laboral/escolar lunes-viernes 8-17h)
# train5['is_active_hours'] = (
#     (train5['day_of_week'] < 5) &  # Lunes-viernes
#     (train5['hour'] >= 8) & 
#     (train5['hour'] <= 17)
# ).astype(int)

# # Últimos 3 meses para evitar problemas de memoria
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']



# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE con is_active_hours (3 meses): {mae:.4f}")
# print(f"MAE anterior (is_daylight): 86.67")

# MAE con is_active_hours (3 meses): 90.4776
# MAE anterior (is_daylight): 86.67

In [21]:
#7mo modelo lgb, agregue variable binaria is_sleeping_hours 22h-6h
#Mae = 98.3152 empeora el modelo , asi que no la saco

# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # Agregar is_sleeping_hours (22h-6h)
# train5['is_sleeping_hours'] = (
#     (train5['hour'] >= 22) | (train5['hour'] <= 6)
# ).astype(int)

# # Últimos 3 meses para evitar problemas de memoria
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# print(f"Dataset original: {len(train5):,}")
# print(f"Últimos 3 meses: {len(sample_df):,}")
# print(f"Porcentaje: {len(sample_df)/len(train5)*100:.1f}%")

# print(f"Sleeping hours distribution:")
# print(sample_df['is_sleeping_hours'].value_counts())

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE con is_active_hours (3 meses): {mae:.4f}")
# print(f"MAE anterior (is_daylight): 86.67")

# Dataset original: 32,963,024
# Últimos 3 meses: 4,674,024
# Porcentaje: 14.2%
# Sleeping hours distribution:
# is_sleeping_hours
# 0    2923632
# 1    1750392
# Name: count, dtype: int64
# Features totales: 46
# Train: 3,739,219 | Test: 934,805
# MAE con is_active_hours (3 meses): 98.3152
# MAE anterior (is_daylight): 86.67

In [22]:
#8vo modelo lgb agregue 3 variables energeticas capacity_per_obs, production_obs_ratio, consumption_obs_ratio
#Mae = 76.2703 mejor bastante el modelo
# Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # Últimos 3 meses
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# # Features energéticas 
# print("🔄 Calculando features energéticas...")

# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
    
#     # Usar installed_capacity como proxy de capacidad productiva
#     capacity = unit_data['installed_capacity'].iloc[0]
    
#     # Ratio de tipos: cuántas observaciones son production vs consumption
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
#     consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
#     production_ratio = production_obs / total_obs if total_obs > 0 else 0
#     consumption_ratio = consumption_obs / total_obs if total_obs > 0 else 0
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_ratio,
#         'consumption_obs_ratio': consumption_ratio
#     })

# energy_df = pd.DataFrame(energy_stats)

# # Merge con datos principales
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# print(f"Últimos 3 meses: {len(sample_df):,}")
# print(f"Nuevas features: capacity_per_obs, production_obs_ratio, consumption_obs_ratio")

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE con features energéticas: {mae:.4f}")
# print(f"MAE anterior: 86.67")

# # BORRAR MEMORIA
# del model, X_train, X_test, y_train, y_test, pred, sample_df, train_data, test_data, energy_df, energy_stats
# import gc
# gc.collect()

# Últimos 3 meses: 4,674,024
# Nuevas features: capacity_per_obs, production_obs_ratio, consumption_obs_ratio
# Features totales: 48
# Train: 3,739,219 | Test: 934,805
# MAE con features energéticas: 76.2703
# MAE anterior: 86.67

In [23]:
#v9 no optimizado para poder usarlo #

# # Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # USAR 4 MESES para calcular ratio, luego reducir a 3
# max_date = train5['datetime'].max()
# cutoff_4m = max_date - pd.DateOffset(months=4)
# cutoff_3m = max_date - pd.DateOffset(months=3)

# sample_4m = train5[train5['datetime'] >= cutoff_4m].copy()
# print("🔄 Calculando ratio production/consumption histórico (30d)...")

# # MÉTODO SIMPLE Y DIRECTO - sin reindexing problemático
# sample_4m['prod_cons_ratio_30d'] = 0.0

# for unit_id in sample_4m['prediction_unit_id'].unique():
#     unit_data = sample_4m[sample_4m['prediction_unit_id'] == unit_id].copy()
#     unit_data = unit_data.sort_values('datetime')
    
#     # Para cada fila, calcular ratio con ventana de 30 días hacia atrás (SIMPLE)
#     for i in range(len(unit_data)):
#         current_time = unit_data.iloc[i]['datetime']
#         start_time = current_time - pd.Timedelta(days=30)
        
#         # Datos en ventana de 30 días
#         window_data = unit_data[
#             (unit_data['datetime'] >= start_time) & 
#             (unit_data['datetime'] < current_time)
#         ]
        
#         if len(window_data) > 0:
#             # Promedios simples
#             prod_mean = window_data[window_data['is_consumption'] == 0]['target'].mean()
#             cons_mean = window_data[window_data['is_consumption'] == 1]['target'].mean()
            
#             # Manejar NaN y división por cero
#             if pd.isna(prod_mean): prod_mean = 0
#             if pd.isna(cons_mean): cons_mean = 1
            
#             ratio = prod_mean / cons_mean if cons_mean != 0 else 0
#             sample_4m.iloc[i, sample_4m.columns.get_loc('prod_cons_ratio_30d')] = ratio

# # Reducir a 3 meses finales
# sample_df = sample_4m[sample_4m['datetime'] >= cutoff_3m].copy()
# print(f"Datos finales (3 meses): {len(sample_df):,}")

# # Features energéticas
# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
#     capacity = unit_data['installed_capacity'].iloc[0]
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0
#     })

# energy_df = pd.DataFrame(energy_stats)
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# # Split y entrenamiento
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# split_idx = int(len(sample_df) * 0.8)
# X_train = sample_df.iloc[:split_idx][feature_cols].dropna()
# y_train = sample_df.iloc[:split_idx].loc[X_train.index, 'target']
# X_test = sample_df.iloc[split_idx:][feature_cols].dropna()
# y_test = sample_df.iloc[split_idx:].loc[X_test.index, 'target']

# print(f"Features: {len(feature_cols)} | Train: {len(X_train):,} | Test: {len(X_test):,}")

# model = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, 
#                          random_state=42, n_jobs=-1, verbose=-1)
# model.fit(X_train, y_train)

# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)
# print(f"✅ MAE con ratio 30d: {mae:.4f}")
# print(f"📊 MAE anterior: 76.27")

# print(f"📈 Ratio stats: min={sample_df['prod_cons_ratio_30d'].min():.3f}, max={sample_df['prod_cons_ratio_30d'].max():.3f}, NaNs={sample_df['prod_cons_ratio_30d'].isna().sum()}")

# # Limpiar memoria
# del model, X_train, X_test, y_train, y_test, pred, sample_df, sample_4m, energy_df, energy_stats
# import gc
# gc.collect()

In [24]:
# #10mo modelo lgb mantengo las 3 anteriores y pruebo  Capacidad instalada per capita (installed_capacity / eic_count)
# # Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # Últimos 3 meses
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# print("🔄 Calculando features energéticas + capacity per capita...")

# # Features energéticas existentes + NUEVA: capacity per capita
# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
    
#     capacity = unit_data['installed_capacity'].iloc[0]
#     eic_count = unit_data['eic_count'].iloc[0]
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
#     consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0,
#         'consumption_obs_ratio': consumption_obs / total_obs if total_obs > 0 else 0,
#         'capacity_per_capita': capacity / eic_count if eic_count > 0 else 0  # NUEVA FEATURE
#     })

# energy_df = pd.DataFrame(energy_stats)
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# print(f"Datos con features energéticas: {len(sample_df):,}")

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)
# print(f"✅ MAE con capacity per capita: {mae:.4f}")
# print(f"📊 MAE anterior: 76.27")

# # LIMPIAR MEMORIA
# del model, X_train, X_test, y_train, y_test, pred, sample_df, train_data, test_data
# del energy_df, energy_stats, feature_importance
# import gc
# gc.collect()
# print("🧹 Memoria liberada")

In [25]:
train5.head()

,row_id,target,datetime,forecast_date,hour,data_block_id,prediction_unit_id,is_business,product_type,county,latitude,longitude,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity,ftemperature,temperature,fdewpoint,dewpoint,fcloudcover_high,cloudcover_high,fcloudcover_low,cloudcover_low,fcloudcover_mid,cloudcover_mid,fcloudcover_total,cloudcover_total,f10_metre_u_wind_component,windspeed_10m,f10_metre_v_wind_component,winddirection_10m,fdirect_solar_radiation,direct_solar_radiation,fsurface_solar_radiation_downwards,shortwave_radiation,diffuse_radiation,fsnowfall,snowfall,ftotal_precipitation,rain,surface_pressure,weather_forecast_hour,is_consumption,day_of_week,month,is_daylight,weekend
6,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.099998,24.200001,45.23,46.32,86.879997,108.0,952.890015,12.681543,12.4,9.783228,9.7,0.023590,11.0,0.002380,10.0,0.001251,0.0,0.026398,12.0,1.840991,4.222222,-3.857846,338.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1009.200012,1.0,0,2,9,0,0
7,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.099998,25.200001,45.23,46.32,86.879997,108.0,952.890015,12.868921,12.3,9.498316,9.6,0.431854,6.0,0.211182,23.0,0.006790,1.0,0.548508,23.0,1.505298,4.027778,-3.590024,337.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1004.200012,1.0,0,2,9,0,0
8,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.400002,22.700001,45.23,46.32,86.879997,108.0,952.890015,15.041773,15.2,11.860376,11.8,0.134674,18.0,0.202515,7.0,0.003906,0.0,0.308930,12.0,3.185351,9.055555,-8.173276,338.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015.000000,1.0,0,2,9,0,0
9,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.400002,23.200001,45.23,46.32,86.879997,108.0,952.890015,14.632105,14.9,11.773584,11.5,0.255188,19.0,0.036774,6.0,0.026245,0.0,0.286194,11.0,3.474780,8.361111,-8.008969,335.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1014.500000,1.0,0,2,9,0,0
10,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.400002,23.700001,45.23,46.32,86.879997,108.0,952.890015,14.480005,12.8,11.581568,10.4,0.338074,12.0,0.037109,4.0,0.016510,1.0,0.368134,8.0,3.211841,5.416667,-7.426206,337.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1014.000000,1.0,0,2,9,0,0


In [26]:
# train5["fsurface_solar_radiation_downwards"].describe()

# count    3.296302e+07
# mean     1.103279e+02
# std      1.714488e+02
# min     -3.258333e-01
# 25%      0.000000e+00
# 50%      5.706424e-01
# 75%      1.429561e+02
# max      8.487144e+02
# Name: fsurface_solar_radiation_downwards, dtype: float64

train5["fsurface_solar_radiation_downwards"].value_counts()

fsurface_solar_radiation_downwards
0.000000      15297128
0.284444         39564
0.142222         35932
0.568889         28742
0.071111         21062
                ...   
169.200272           2
320.213043           2
48.876667            2
42.334446            2
268.698120           2
Name: count, Length: 1291576, dtype: int64

In [27]:
# #11vo modelo lgb 4 feature electrica y pruebo panel_efficiency
# # ✅ MAE con panel efficiency: 85.0309
# # Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # Últimos 3 meses
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# print("🔄 Calculando features energéticas + capacity per capita + panel efficiency...")

# # Features energéticas existentes + NUEVAS: capacity per capita + panel efficiency
# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
    
#     capacity = unit_data['installed_capacity'].iloc[0]
#     eic_count = unit_data['eic_count'].iloc[0]
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
#     consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
#     # Calcular eficiencia de paneles (production vs solar_radiation cuando is_consumption=False)
#     production_data = unit_data[unit_data['is_consumption'] == 0]
#     if len(production_data) > 0 and 'fsurface_solar_radiation_downwards' in production_data.columns:
#         solar_rad = production_data['fsurface_solar_radiation_downwards']
#         production = production_data['target']
#         # Filtrar valores válidos (solar > 0 para evitar división por 0)
#         valid_mask = (solar_rad > 0) & (production >= 0)
#         if valid_mask.sum() > 0:
#             panel_efficiency = (production[valid_mask] / solar_rad[valid_mask]).mean()
#         else:
#             panel_efficiency = 0
#     else:
#         panel_efficiency = 0
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0,
#         'consumption_obs_ratio': consumption_obs / total_obs if total_obs > 0 else 0,
#         'capacity_per_capita': capacity / eic_count if eic_count > 0 else 0,
#         'panel_efficiency': panel_efficiency  # NUEVA FEATURE
#     })

# energy_df = pd.DataFrame(energy_stats)
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# print(f"Datos con features energéticas: {len(sample_df):,}")

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)
# print(f"✅ MAE con panel efficiency: {mae:.4f}")
# print(f"📊 MAE anterior: 72.3572")

# # Verificar nuevas features
# print(f"📈 Panel efficiency stats:")
# print(f"   - Min: {sample_df['panel_efficiency'].min():.4f}")
# print(f"   - Max: {sample_df['panel_efficiency'].max():.4f}")
# print(f"   - Mean: {sample_df['panel_efficiency'].mean():.4f}")
# print(f"   - NaN count: {sample_df['panel_efficiency'].isna().sum()}")

# # LIMPIAR MEMORIA
# del model, X_train, X_test, y_train, y_test, pred, sample_df, train_data, test_data
# del energy_df, energy_stats
# import gc
# gc.collect()
# print("🧹 Memoria liberada")

In [28]:
# #12vo modelo lgb , lo anterior pero ahora efficiency panel es log
# # ✅ MAE con panel efficiency LOG: 85.3200
# # 📊 MAE anterior: 72.3572

# # Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # Últimos 3 meses
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# print("🔄 Calculando features energéticas + capacity per capita + panel efficiency LOG...")

# # Features energéticas existentes + NUEVAS: capacity per capita + panel efficiency LOG
# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
    
#     capacity = unit_data['installed_capacity'].iloc[0]
#     eic_count = unit_data['eic_count'].iloc[0]
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
#     consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
#     # Calcular eficiencia de paneles LOG (production vs solar_radiation cuando is_consumption=False)
#     production_data = unit_data[unit_data['is_consumption'] == 0]
#     if len(production_data) > 0 and 'fsurface_solar_radiation_downwards' in production_data.columns:
#         solar_rad = production_data['fsurface_solar_radiation_downwards']
#         production = production_data['target']
#         # Filtrar valores válidos (solar > 0 para evitar división por 0)
#         valid_mask = (solar_rad > 0) & (production >= 0)
#         if valid_mask.sum() > 0:
#             efficiency_ratio = (production[valid_mask] / solar_rad[valid_mask]).mean()
#             # Aplicar log pero manejando valores <= 1
#             if efficiency_ratio > 1:
#                 panel_efficiency_log = np.log(efficiency_ratio)
#             elif efficiency_ratio > 0:
#                 panel_efficiency_log = -np.log(1/efficiency_ratio)  # log inverso para valores < 1
#             else:
#                 panel_efficiency_log = 0
#         else:
#             panel_efficiency_log = 0
#     else:
#         panel_efficiency_log = 0
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0,
#         'consumption_obs_ratio': consumption_obs / total_obs if total_obs > 0 else 0,
#         'capacity_per_capita': capacity / eic_count if eic_count > 0 else 0,
#         'panel_efficiency_log': panel_efficiency_log  # NUEVA FEATURE CON LOG
#     })

# energy_df = pd.DataFrame(energy_stats)
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# print(f"Datos con features energéticas: {len(sample_df):,}")

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)
# print(f"✅ MAE con panel efficiency LOG: {mae:.4f}")
# print(f"📊 MAE anterior: 72.3572")

# # Verificar nuevas features
# print(f"📈 Capacity per capita stats:")
# print(f"   - Min: {sample_df['capacity_per_capita'].min():.4f}")
# print(f"   - Max: {sample_df['capacity_per_capita'].max():.4f}")
# print(f"   - Mean: {sample_df['capacity_per_capita'].mean():.4f}")
# print(f"   - NaN count: {sample_df['capacity_per_capita'].isna().sum()}")

# print(f"📈 Panel efficiency LOG stats:")
# print(f"   - Min: {sample_df['panel_efficiency_log'].min():.4f}")
# print(f"   - Max: {sample_df['panel_efficiency_log'].max():.4f}")
# print(f"   - Mean: {sample_df['panel_efficiency_log'].mean():.4f}")
# print(f"   - NaN count: {sample_df['panel_efficiency_log'].isna().sum()}")

# # LIMPIAR MEMORIA
# del model, X_train, X_test, y_train, y_test, pred, sample_df, train_data, test_data
# del energy_df, energy_stats
# import gc
# gc.collect()
# print("🧹 Memoria liberada")

In [29]:
# #13vo modelo probando volatilidad de precio de electricidad 7 dias
# # ✅ MAE con electricity price volatility: 73.2246
# # 📊 MAE anterior: 72.3572
# # Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # USAR 120 días para calcular volatilidad, luego reducir a 90
# max_date = train5['datetime'].max()
# cutoff_120d = max_date - pd.DateOffset(days=120)  # 120 días para calcular volatilidad
# cutoff_90d = max_date - pd.DateOffset(days=90)    # 90 días finales

# # Datos de 120 días para calcular volatilidad
# sample_120d = train5[train5['datetime'] >= cutoff_120d].copy()

# print("🔄 Calculando volatilidad de precios (7d rolling) con 120 días...")

# # Calcular volatilidad de electricity prices (rolling std 7 días)
# sample_120d['electricity_price_volatility_7d'] = 0.0

# for unit_id in sample_120d['prediction_unit_id'].unique():
#     unit_data = sample_120d[sample_120d['prediction_unit_id'] == unit_id].copy()
#     unit_data = unit_data.sort_values('datetime')
    
#     if 'euros_per_mwh' in unit_data.columns and len(unit_data) > 0:
#         # Rolling std de 7 días para electricity prices
#         unit_data = unit_data.set_index('datetime')
#         price_vol = unit_data['euros_per_mwh'].rolling('7D', min_periods=1).std()
        
#         # Manejar NaN y resetear index
#         price_vol = price_vol.fillna(0)
#         unit_data = unit_data.reset_index()
        
#         # Asignar volatilidad al dataframe principal
#         mask = sample_120d['prediction_unit_id'] == unit_id
#         sample_120d.loc[mask, 'electricity_price_volatility_7d'] = price_vol.values

# # AHORA reducir a últimos 90 días (con volatilidad ya calculada)
# sample_df = sample_120d[sample_120d['datetime'] >= cutoff_90d].copy()
# print(f"Datos finales (90 días con volatilidad): {len(sample_df):,}")

# print("🔄 Calculando features energéticas + capacity per capita...")

# # Features energéticas existentes + capacity per capita (SIN panel_efficiency)
# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
    
#     capacity = unit_data['installed_capacity'].iloc[0]
#     eic_count = unit_data['eic_count'].iloc[0]
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
#     consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0,
#         'consumption_obs_ratio': consumption_obs / total_obs if total_obs > 0 else 0,
#         'capacity_per_capita': capacity / eic_count if eic_count > 0 else 0
#     })

# energy_df = pd.DataFrame(energy_stats)
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# print(f"Datos con features energéticas + volatilidad: {len(sample_df):,}")

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)
# print(f"✅ MAE con electricity price volatility: {mae:.4f}")
# print(f"📊 MAE anterior: 72.3572")

# # Verificar nuevas features
# print(f"📈 Electricity price volatility 7d stats:")
# print(f"   - Min: {sample_df['electricity_price_volatility_7d'].min():.4f}")
# print(f"   - Max: {sample_df['electricity_price_volatility_7d'].max():.4f}")
# print(f"   - Mean: {sample_df['electricity_price_volatility_7d'].mean():.4f}")
# print(f"   - NaN count: {sample_df['electricity_price_volatility_7d'].isna().sum()}")

# # LIMPIAR MEMORIA
# del model, X_train, X_test, y_train, y_test, pred, sample_df, sample_120d, train_data, test_data
# del energy_df, energy_stats
# import gc
# gc.collect()
# print("🧹 Memoria liberada")

In [30]:
# #14vo modelo ademas de electricity_volatility pruebo con gas
# # ✅ MAE con electricity + gas price volatility: 70.6517
# # 📊 MAE anterior: 73.2246

# # Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # USAR 120 días para calcular volatilidad, luego reducir a 90
# max_date = train5['datetime'].max()
# cutoff_120d = max_date - pd.DateOffset(days=120)  # 120 días para calcular volatilidad
# cutoff_90d = max_date - pd.DateOffset(days=90)    # 90 días finales

# # Datos de 120 días para calcular volatilidad
# sample_120d = train5[train5['datetime'] >= cutoff_120d].copy()

# print("🔄 Calculando volatilidad de electricity y gas prices (7d rolling) con 120 días...")

# # Calcular volatilidad de electricity y gas prices (rolling std 7 días)
# sample_120d['electricity_price_volatility_7d'] = 0.0
# sample_120d['gas_price_volatility_7d'] = 0.0

# for unit_id in sample_120d['prediction_unit_id'].unique():
#     unit_data = sample_120d[sample_120d['prediction_unit_id'] == unit_id].copy()
#     unit_data = unit_data.sort_values('datetime')
    
#     if len(unit_data) > 0:
#         unit_data_indexed = unit_data.set_index('datetime')
        
#         # Volatilidad electricity prices
#         if 'euros_per_mwh' in unit_data.columns:
#             elec_vol = unit_data_indexed['euros_per_mwh'].rolling('7D', min_periods=1).std().fillna(0)
#         else:
#             elec_vol = pd.Series(0, index=unit_data_indexed.index)
        
#         # Volatilidad gas prices
#         if 'lowest_price_per_mwh' in unit_data.columns:
#             gas_vol = unit_data_indexed['lowest_price_per_mwh'].rolling('7D', min_periods=1).std().fillna(0)
#         else:
#             gas_vol = pd.Series(0, index=unit_data_indexed.index)
        
#         # Asignar volatilidades al dataframe principal usando iloc
#         mask = sample_120d['prediction_unit_id'] == unit_id
#         indices = sample_120d[mask].index
        
#         sample_120d.loc[indices, 'electricity_price_volatility_7d'] = elec_vol.values
#         sample_120d.loc[indices, 'gas_price_volatility_7d'] = gas_vol.values

# # AHORA reducir a últimos 90 días (conservando TODAS las columnas)
# sample_df = sample_120d[sample_120d['datetime'] >= cutoff_90d].copy()
# print(f"Datos finales (90 días con volatilidades): {len(sample_df):,}")

# print("🔄 Calculando features energéticas + capacity per capita...")

# # Features energéticas existentes + capacity per capita
# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
    
#     capacity = unit_data['installed_capacity'].iloc[0]
#     eic_count = unit_data['eic_count'].iloc[0]
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
#     consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0,
#         'consumption_obs_ratio': consumption_obs / total_obs if total_obs > 0 else 0,
#         'capacity_per_capita': capacity / eic_count if eic_count > 0 else 0
#     })

# energy_df = pd.DataFrame(energy_stats)
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# print(f"Datos con features energéticas + volatilidades: {len(sample_df):,}")


# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)
# print(f"✅ MAE con electricity + gas price volatility: {mae:.4f}")
# print(f"📊 MAE anterior: 73.2246")

# # Verificar nuevas features

# if 'gas_price_volatility_7d' in sample_df.columns:
#     print(f"📈 Gas price volatility 7d stats:")
#     print(f"   - Min: {sample_df['gas_price_volatility_7d'].min():.4f}")
#     print(f"   - Max: {sample_df['gas_price_volatility_7d'].max():.4f}")
#     print(f"   - Mean: {sample_df['gas_price_volatility_7d'].mean():.4f}")
#     print(f"   - NaN count: {sample_df['gas_price_volatility_7d'].isna().sum()}")

# # LIMPIAR MEMORIA
# del model, X_train, X_test, y_train, y_test, pred, sample_df, sample_120d, train_data, test_data
# del energy_df, energy_stats
# import gc
# gc.collect()
# print("🧹 Memoria liberada")

In [31]:
# CREAR TRAIN6 con 150 días y todas las features consolidadas
print("🔄 Creando train6 con 150 días...")

# Usar train5 como base
train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# Últimos 150 días para train6
max_date = train5['datetime'].max()
cutoff_150d = max_date - pd.DateOffset(days=150)
train6 = train5[train5['datetime'] >= cutoff_150d].copy()

print(f"📊 train6 creado: {len(train6):,} filas ({len(train6['prediction_unit_id'].unique())} units)")

print("🔄 Calculando features energéticas...")

# Features energéticas
energy_stats = []
for unit_id in train6['prediction_unit_id'].unique():
    unit_data = train6[train6['prediction_unit_id'] == unit_id]
    
    capacity = unit_data['installed_capacity'].iloc[0]
    eic_count = unit_data['eic_count'].iloc[0]
    total_obs = len(unit_data)
    production_obs = len(unit_data[unit_data['is_consumption'] == 0])
    consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
    energy_stats.append({
        'prediction_unit_id': unit_id,
        'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
        'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0,
        'consumption_obs_ratio': consumption_obs / total_obs if total_obs > 0 else 0,
        'capacity_per_capita': capacity / eic_count if eic_count > 0 else 0
    })

energy_df = pd.DataFrame(energy_stats)
train6 = train6.merge(energy_df, on='prediction_unit_id', how='left')

print("🔄 Calculando volatilidades de precios (7d rolling)...")

# Volatilidades de precios (7d rolling)
train6['electricity_price_volatility_7d'] = 0.0
train6['gas_price_volatility_7d'] = 0.0

for unit_id in train6['prediction_unit_id'].unique():
    unit_data = train6[train6['prediction_unit_id'] == unit_id].copy()
    unit_data = unit_data.sort_values('datetime')
    
    if len(unit_data) > 0:
        unit_data_indexed = unit_data.set_index('datetime')
        
        # Volatilidad electricity prices
        if 'euros_per_mwh' in unit_data.columns:
            elec_vol = unit_data_indexed['euros_per_mwh'].rolling('7D', min_periods=1).std().fillna(0)
        else:
            elec_vol = pd.Series(0, index=unit_data_indexed.index)
        
        # Volatilidad gas prices
        if 'lowest_price_per_mwh' in unit_data.columns:
            gas_vol = unit_data_indexed['lowest_price_per_mwh'].rolling('7D', min_periods=1).std().fillna(0)
        else:
            gas_vol = pd.Series(0, index=unit_data_indexed.index)
        
        # Asignar volatilidades
        mask = train6['prediction_unit_id'] == unit_id
        indices = train6[mask].index
        
        train6.loc[indices, 'electricity_price_volatility_7d'] = elec_vol.values
        train6.loc[indices, 'gas_price_volatility_7d'] = gas_vol.values

print("✅ train6 creado con todas las features:")
print(f"   📊 Datos: {len(train6):,} filas")
print(f"   📊 Período: {train6['datetime'].min()} a {train6['datetime'].max()}")
print(f"   🔧 Features energéticas: 4 (capacity_per_obs, production_obs_ratio, consumption_obs_ratio, capacity_per_capita)")
print(f"   📈 Volatilidades: 2 (electricity_price_volatility_7d, gas_price_volatility_7d)")

# Verificar features creadas
feature_check = [
    'capacity_per_obs', 'production_obs_ratio', 'consumption_obs_ratio', 
    'capacity_per_capita', 'electricity_price_volatility_7d', 'gas_price_volatility_7d'
]

print("\n🔍 Verificación de features:")
for feature in feature_check:
    if feature in train6.columns:
        print(f"   ✅ {feature}: OK")
    else:
        print(f"   ❌ {feature}: FALTA")

# Estadísticas básicas de las nuevas features
print(f"\n📈 Estadísticas rápidas:")
for feature in feature_check:
    if feature in train6.columns:
        print(f"   {feature}: min={train6[feature].min():.3f}, max={train6[feature].max():.3f}, mean={train6[feature].mean():.3f}")

print(f"\n✨ train6 listo para usar con {len(train6.columns)} columnas totales")

# Limpiar memoria temporal
del energy_df, energy_stats
import gc
gc.collect()
print("🧹 Memoria temporal liberada")

🔄 Creando train6 con 150 días...
📊 train6 creado: 7,821,024 filas (68 units)
🔄 Calculando features energéticas...
🔄 Calculando volatilidades de precios (7d rolling)...
✅ train6 creado con todas las features:
   📊 Datos: 7,821,024 filas
   📊 Período: 2022-12-31 10:00:00 a 2023-05-30 10:00:00
   🔧 Features energéticas: 4 (capacity_per_obs, production_obs_ratio, consumption_obs_ratio, capacity_per_capita)
   📈 Volatilidades: 2 (electricity_price_volatility_7d, gas_price_volatility_7d)

🔍 Verificación de features:
   ✅ capacity_per_obs: OK
   ✅ production_obs_ratio: OK
   ✅ consumption_obs_ratio: OK
   ✅ capacity_per_capita: OK
   ✅ electricity_price_volatility_7d: OK
   ✅ gas_price_volatility_7d: OK

📈 Estadísticas rápidas:
   capacity_per_obs: min=0.000, max=0.091, mean=0.014
   production_obs_ratio: min=0.500, max=0.500, mean=0.500
   consumption_obs_ratio: min=0.500, max=0.500, mean=0.500
   capacity_per_capita: min=0.438, max=213.750, mean=26.458
   electricity_price_volatility_7d: mi

In [32]:
# # Modelo #16: Lag 24h de target por prediction_unit_id
# #  últimos 90 días 

# print("🔄 Modelo #16: Lag 24h con últimos 90 días...")

# # Filtrar últimos 90 días de train6
# max_date = train6['datetime'].max()
# cutoff_90d = max_date - pd.DateOffset(days=90)
# sample_90d = train6[train6['datetime'] >= cutoff_90d].copy()

# print(f"📊 Dataset 90 días: {len(sample_90d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])

# # Crear lag 24h por prediction_unit_id
# sample_90d['target_lag_24h'] = sample_90d.groupby('prediction_unit_id')['target'].shift(24)

# # Stats del lag
# valid_lags = sample_90d['target_lag_24h'].notna().sum()
# print(f"📈 Lag 24h válidos: {valid_lags:,} ({valid_lags/len(sample_90d)*100:.1f}%)")

# # Limpiar NaN en lag
# train_clean = sample_90d.dropna(subset=['target_lag_24h'])
# print(f"📊 Datos finales: {len(train_clean):,}")

# # Features (incluye nueva lag)
# feature_cols = [col for col in train_clean.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split 80/20
# split_idx = int(len(train_clean) * 0.8)
# train_data = train_clean.iloc[:split_idx]
# test_data = train_clean.iloc[split_idx:]

# X_train = train_data[feature_cols]
# y_train = train_data['target']
# X_test = test_data[feature_cols]
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con lag 24h (90 días): {mae:.4f}")
# print(f"📊 MAE anterior: 70.6517")
# print(f"📈 Cambio: {mae - 70.6517:+.4f}")



In [33]:
# # 17vo modelo Lag 48h de target por prediction_unit_id , no va
# #✅ MAE con lag 48h (90 días): 104.3003
# #📊 MAE anterior: 70.6517
# #  últimos 90 días 

# print("🔄 Modelo #17: Lag 48h con últimos 90 días...")

# # Filtrar últimos 90 días de train6
# max_date = train6['datetime'].max()
# cutoff_90d = max_date - pd.DateOffset(days=90)
# sample_90d = train6[train6['datetime'] >= cutoff_90d].copy()

# print(f"📊 Dataset 90 días: {len(sample_90d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])

# # Crear lag 24h por prediction_unit_id
# sample_90d['target_lag_48h'] = sample_90d.groupby('prediction_unit_id')['target'].shift(24)

# # Stats del lag
# valid_lags = sample_90d['target_lag_48h'].notna().sum()
# print(f"📈 Lag 24h válidos: {valid_lags:,} ({valid_lags/len(sample_90d)*100:.1f}%)")

# # Limpiar NaN en lag
# train_clean = sample_90d.dropna(subset=['target_lag_48h'])
# print(f"📊 Datos finales: {len(train_clean):,}")

# # Features (incluye nueva lag)
# feature_cols = [col for col in train_clean.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split 80/20
# split_idx = int(len(train_clean) * 0.8)
# train_data = train_clean.iloc[:split_idx]
# test_data = train_clean.iloc[split_idx:]

# X_train = train_data[feature_cols]
# y_train = train_data['target']
# X_test = test_data[feature_cols]
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con lag 48h (90 días): {mae:.4f}")
# print(f"📊 MAE anterior: 70.6517")
# print(f"📈 Cambio: {mae - 70.6517:+.4f}")


In [34]:
# # Modelo #18: Cyclical encoding de hora (hour_sin, hour_cos)
# ✅ MAE con hour_sin/cos: 122.4835 # algo esta pasado ver como corregirlo
# 📊 MAE anterior: 70.6517

# # Usando últimos 90 días

# print("🔄 Modelo #18: Agregando hour_sin y hour_cos...")

# # Filtrar últimos 90 días de train6
# max_date = train6['datetime'].max()
# cutoff_90d = max_date - pd.DateOffset(days=90)
# sample_90d = train6[train6['datetime'] >= cutoff_90d].copy()

# print(f"📊 Dataset 90 días: {len(sample_90d):,} registros")

# # Crear cyclical encoding de hora
# sample_90d['hour_sin'] = np.sin(2 * np.pi * sample_90d['hour'] / 24)
# sample_90d['hour_cos'] = np.cos(2 * np.pi * sample_90d['hour'] / 24)

# print(f"📈 Hour_sin range: [{sample_90d['hour_sin'].min():.3f}, {sample_90d['hour_sin'].max():.3f}]")
# print(f"📈 Hour_cos range: [{sample_90d['hour_cos'].min():.3f}, {sample_90d['hour_cos'].max():.3f}]")

# # Features (incluye nuevas cyclical)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols]
# y_train = train_data['target']
# X_test = test_data[feature_cols]
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con hour_sin/cos: {mae:.4f}")
# print(f"📊 MAE anterior: 70.6517")



In [35]:
# # Modelo #19: Cyclical encoding de semana (week_sin, week_cos)
# #✅ MAE con week_sin/cos: 118.3710 los temporales , de la forma que los planteo no estan funcionando
# #📊 MAE anterior: 70.6517
# # Usando últimos 90 días

# print("🔄 Modelo #19: Agregando week_sin y week_cos...")

# # Filtrar últimos 90 días de train6
# max_date = train6['datetime'].max()
# cutoff_90d = max_date - pd.DateOffset(days=90)
# sample_90d = train6[train6['datetime'] >= cutoff_90d].copy()

# print(f"📊 Dataset 90 días: {len(sample_90d):,} registros")

# # Crear cyclical encoding de semana del año
# sample_90d['week_of_year'] = sample_90d['datetime'].dt.isocalendar().week
# sample_90d['week_sin'] = np.sin(2 * np.pi * sample_90d['week_of_year'] / 52)
# sample_90d['week_cos'] = np.cos(2 * np.pi * sample_90d['week_of_year'] / 52)

# print(f"📈 Week range: [{sample_90d['week_of_year'].min()}, {sample_90d['week_of_year'].max()}]")
# print(f"📈 Week_sin range: [{sample_90d['week_sin'].min():.3f}, {sample_90d['week_sin'].max():.3f}]")
# print(f"📈 Week_cos range: [{sample_90d['week_cos'].min():.3f}, {sample_90d['week_cos'].max():.3f}]")

# # Features (incluye nuevas cyclical de semana)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols]
# y_train = train_data['target']
# X_test = test_data[feature_cols]
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con week_sin/cos: {mae:.4f}")
# print(f"📊 MAE anterior: 70.6517")


In [36]:
# # Modelo #20: Cambios día a día (delta vs día anterior) 'temperature', 'euros_per_mwh', 'lowest_price_per_mwh', 'direct_solar_radiation'
# # ✅ MAE con deltas weather 24h: 124.1003 algo mal estoy haciendo o el overfit es grande por 90dias ,capaz con mas dias menos overfit
# # 📊 MAE anterior: 70.6517
# # Usando últimos 90 días

# print("🔄 Modelo #20: Agregando deltas día a día...")

# # Filtrar últimos 90 días de train6
# max_date = train6['datetime'].max()
# cutoff_90d = max_date - pd.DateOffset(days=90)
# sample_90d = train6[train6['datetime'] >= cutoff_90d].copy()

# print(f"📊 Dataset 90 días: {len(sample_90d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])

# # Crear deltas día a día de weather features 
# weather_cols = ['temperature', 'euros_per_mwh', 'lowest_price_per_mwh', 'direct_solar_radiation']

# for col in weather_cols:
#     if col in sample_90d.columns:
#         # Delta de 24h hacia atrás por prediction_unit_id
#         sample_90d[f'{col}_delta_24h'] = (sample_90d[col] - 
#                                          sample_90d.groupby('prediction_unit_id')[col].shift(24))
        
#         # Stats del delta
#         valid_deltas = sample_90d[f'{col}_delta_24h'].notna().sum()
#         print(f"📈 {col}_delta_24h válidos: {valid_deltas:,}")

# # Limpiar NaN en deltas (al menos uno debe ser válido)
# delta_cols = [f'{col}_delta_24h' for col in weather_cols if col in sample_90d.columns]
# sample_90d = sample_90d.dropna(subset=delta_cols, how='all')
# print(f"📊 Datos finales: {len(sample_90d):,}")

# # Features (incluye nuevos deltas)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols].fillna(0)
# y_train = train_data['target']
# X_test = test_data[feature_cols].fillna(0)
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con deltas weather 24h: {mae:.4f}")
# print(f"📊 MAE anterior: 70.6517")

In [37]:
# # Modelo #21: Media móvil de target por prediction_unit_id
# # ✅ MAE con media móvil 7d: 56.1320 MMM data leakage por como estan los datos , veo como lo cambio
# # 📊 MAE anterior: 70.6517
# # Usando últimos 90 días

# print("🔄 Modelo #21: Agregando media móvil de target...")

# # Filtrar últimos 90 días de train6
# max_date = train6['datetime'].max()
# cutoff_90d = max_date - pd.DateOffset(days=90)
# sample_90d = train6[train6['datetime'] >= cutoff_90d].copy()

# print(f"📊 Dataset 90 días: {len(sample_90d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])

# # Crear media móvil de target por prediction_unit_id (ventana 7 días)
# sample_90d['target_ma_7d'] = (sample_90d.groupby('prediction_unit_id')['target']
#                               .shift(1)  # Evitar data leakage
#                               .rolling(window=168, min_periods=24)  # 7 días = 168 horas
#                               .mean())

# # Stats de la media móvil
# valid_ma = sample_90d['target_ma_7d'].notna().sum()
# print(f"📈 Target MA 7d válidos: {valid_ma:,} ({valid_ma/len(sample_90d)*100:.1f}%)")

# # Limpiar NaN en media móvil
# sample_90d = sample_90d.dropna(subset=['target_ma_7d'])
# print(f"📊 Datos finales: {len(sample_90d):,}")

# # Features (incluye nueva media móvil)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols]
# y_train = train_data['target']
# X_test = test_data[feature_cols]
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con media móvil 7d: {mae:.4f}")
# print(f"📊 MAE anterior: 70.6517")


In [38]:
# # Modelo #22: Media móvil de target por prediction_unit_id lag 5 dias
# # ✅ MAE con media móvil 7d (lag 5d): 61.2636
# # 📊 MAE anterior: 70.6517
# # Usando 120 días para calcular, reducir a 90 días

# print("🔄 Modelo #22: Media móvil con datos suficientes...")

# # USAR 120 días para calcular media móvil, luego reducir a 90
# max_date = train6['datetime'].max()
# cutoff_120d = max_date - pd.DateOffset(days=120)  # 120 días para calcular
# cutoff_90d = max_date - pd.DateOffset(days=90)    # 90 días finales

# # Datos de 120 días para calcular media móvil
# sample_120d = train6[train6['datetime'] >= cutoff_120d].copy()
# print(f"📊 Dataset 120 días para cálculo: {len(sample_120d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_120d = sample_120d.sort_values(['prediction_unit_id', 'datetime'])

# # Crear media móvil de target por prediction_unit_id (ventana 7 días, retraso 5 días)
# sample_120d['target_ma_7d_lag5d'] = (sample_120d.groupby('prediction_unit_id')['target']
#                                    .shift(120)  # 5 días = 120 horas
#                                    .rolling(window=168, min_periods=168)  # 7 días = 168 horas
#                                    .mean())

# # AHORA reducir a últimos 90 días
# sample_90d = sample_120d[sample_120d['datetime'] >= cutoff_90d].copy()

# # Stats de la media móvil
# valid_ma = sample_90d['target_ma_7d_lag5d'].notna().sum()
# print(f"📊 Datos finales (90 días): {len(sample_90d):,} registros")
# print(f"📈 Target MA 7d (lag 5d) válidos: {valid_ma:,} ({valid_ma/len(sample_90d)*100:.1f}%)")

# # Limpiar NaN en media móvil
# sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
# print(f"📊 Datos después de limpiar NaN: {len(sample_90d):,}")

# # Features (incluye nueva media móvil)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols]
# y_train = train_data['target']
# X_test = test_data[feature_cols]
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con media móvil 7d (lag 5d): {mae:.4f}")
# print(f"📊 MAE anterior: 70.6517")


In [39]:
# # Modelo #23: Media móvil de target por prediction_unit_id lag 5 dias
# #✅ MAE con ambas media móvil 7d y 3d (lag 5d): 64.0943
# #📊 MAE anterior: 61.2636
# # Usando 120 días para calcular, reducir a 90 días

# print("🔄 Modelo #23: Media móvil con datos suficientes...")

# # USAR 120 días para calcular media móvil, luego reducir a 90
# max_date = train6['datetime'].max()
# cutoff_120d = max_date - pd.DateOffset(days=120)  # 120 días para calcular
# cutoff_90d = max_date - pd.DateOffset(days=90)    # 90 días finales

# # Datos de 120 días para calcular media móvil
# sample_120d = train6[train6['datetime'] >= cutoff_120d].copy()
# print(f"📊 Dataset 120 días para cálculo: {len(sample_120d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_120d = sample_120d.sort_values(['prediction_unit_id', 'datetime'])

# # Crear media móvil de target por prediction_unit_id (ventana 7 días, retraso 5 días)
# sample_120d['target_ma_7d_lag5d'] = (sample_120d.groupby('prediction_unit_id')['target']
#                                    .shift(120)  # 5 días = 120 horas
#                                    .rolling(window=168, min_periods=168)  # 7 días = 168 horas
#                                    .mean())

# sample_120d['target_ma_3d_lag5d'] = (sample_120d.groupby('prediction_unit_id')['target']
#                                    .shift(120)  # 5 días = 120 horas
#                                    .rolling(window=72, min_periods=72)  # 3 días = 72 horas
#                                    .mean())


# # AHORA reducir a últimos 90 días
# sample_90d = sample_120d[sample_120d['datetime'] >= cutoff_90d].copy()

# # Stats de la media móvil
# valid_ma = sample_90d['target_ma_7d_lag5d'].notna().sum()
# print(f"📊 Datos finales (90 días): {len(sample_90d):,} registros")
# print(f"📈 Target MA 7d (lag 5d) válidos: {valid_ma:,} ({valid_ma/len(sample_90d)*100:.1f}%)")

# # Stats de la media móvil
# valid_ma1 = sample_90d['target_ma_3d_lag5d'].notna().sum()
# print(f"📊 Datos finales (90 días): {len(sample_90d):,} registros")
# print(f"📈 Target MA 3d (lag 5d) válidos: {valid_ma1:,} ({valid_ma1/len(sample_90d)*100:.1f}%)")


# # Limpiar NaN en media móvil
# sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
# print(f"📊 Datos después de limpiar NaN: {len(sample_90d):,}")

# sample_90d = sample_90d.dropna(subset=['target_ma_3d_lag5d'])
# print(f"📊 Datos después de limpiar NaN: {len(sample_90d):,}")

# # Features (incluye nueva media móvil)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols]
# y_train = train_data['target']
# X_test = test_data[feature_cols]
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con media móvil 3d (lag 5d): {mae:.4f}")
# print(f"📊 MAE anterior: 61.2636")

In [40]:
# # Modelo #24: Media móvil de target por prediction_unit_id lag 5 dias
# #✅ MAE con media móvil 3d (lag 5d): 63.5897
# #📊 MAE anterior: 61.2636
# # Usando 120 días para calcular, reducir a 90 días

# print("🔄 Modelo #24: Media móvil 3d...")

# # USAR 120 días para calcular media móvil, luego reducir a 90
# max_date = train6['datetime'].max()
# cutoff_120d = max_date - pd.DateOffset(days=120)  # 120 días para calcular
# cutoff_90d = max_date - pd.DateOffset(days=90)    # 90 días finales

# # Datos de 120 días para calcular media móvil
# sample_120d = train6[train6['datetime'] >= cutoff_120d].copy()
# print(f"📊 Dataset 120 días para cálculo: {len(sample_120d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_120d = sample_120d.sort_values(['prediction_unit_id', 'datetime'])

# # Crear media móvil de target por prediction_unit_id (ventana 7 días, retraso 5 días)
# sample_120d['target_ma_3d_lag5d'] = (sample_120d.groupby('prediction_unit_id')['target']
#                                    .shift(120)  # 5 días = 120 horas
#                                    .rolling(window=72, min_periods=72)  # 3 días = 72 horas
#                                    .mean())

# # AHORA reducir a últimos 90 días
# sample_90d = sample_120d[sample_120d['datetime'] >= cutoff_90d].copy()

# # Stats de la media móvil
# valid_ma = sample_90d['target_ma_3d_lag5d'].notna().sum()
# print(f"📊 Datos finales (90 días): {len(sample_90d):,} registros")
# print(f"📈 Target MA 3d (lag 5d) válidos: {valid_ma:,} ({valid_ma/len(sample_90d)*100:.1f}%)")

# # Limpiar NaN en media móvil
# sample_90d = sample_90d.dropna(subset=['target_ma_3d_lag5d'])
# print(f"📊 Datos después de limpiar NaN: {len(sample_90d):,}")

# # Features (incluye nueva media móvil)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols]
# y_train = train_data['target']
# X_test = test_data[feature_cols]
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con media móvil 3d (lag 5d): {mae:.4f}")
# print(f"📊 MAE anterior: 61.2636")


In [41]:
# # Modelo #25: Media móvil de target por prediction_unit_id lag 5 dias
# #✅ MAE con media móvil 5d (lag 5d): 71.2301
# #📊 MAE anterior: 61.2636
# # Usando 120 días para calcular, reducir a 90 días

# print("🔄 Modelo #25: Media móvil 5d...")

# # USAR 120 días para calcular media móvil, luego reducir a 90
# max_date = train6['datetime'].max()
# cutoff_120d = max_date - pd.DateOffset(days=120)  # 120 días para calcular
# cutoff_90d = max_date - pd.DateOffset(days=90)    # 90 días finales

# # Datos de 120 días para calcular media móvil
# sample_120d = train6[train6['datetime'] >= cutoff_120d].copy()
# print(f"📊 Dataset 120 días para cálculo: {len(sample_120d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_120d = sample_120d.sort_values(['prediction_unit_id', 'datetime'])

# # Crear media móvil de target por prediction_unit_id (ventana 7 días, retraso 5 días)
# sample_120d['target_ma_5d_lag5d'] = (sample_120d.groupby('prediction_unit_id')['target']
#                                    .shift(120)  # 5 días = 120 horas
#                                    .rolling(window=120, min_periods=120)  # 3 días = 72 horas
#                                    .mean())

# # AHORA reducir a últimos 90 días
# sample_90d = sample_120d[sample_120d['datetime'] >= cutoff_90d].copy()

# # Stats de la media móvil
# valid_ma = sample_90d['target_ma_5d_lag5d'].notna().sum()
# print(f"📊 Datos finales (90 días): {len(sample_90d):,} registros")
# print(f"📈 Target MA 5d (lag 5d) válidos: {valid_ma:,} ({valid_ma/len(sample_90d)*100:.1f}%)")

# # Limpiar NaN en media móvil
# sample_90d = sample_90d.dropna(subset=['target_ma_5d_lag5d'])
# print(f"📊 Datos después de limpiar NaN: {len(sample_90d):,}")

# # Features (incluye nueva media móvil)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols]
# y_train = train_data['target']
# X_test = test_data[feature_cols]
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con media móvil 5d (lag 5d): {mae:.4f}")
# print(f"📊 MAE anterior: 61.2636")


In [42]:
# Crear train7: train6 + media móvil 7d lag 5d + sin weather histórico
# train6 tiene 150 días, primeros días tendrán NaN en MA

print("🔄 Creando train7: train6 + target_ma_7d_lag5d + sin weather histórico...")

# Copiar train6 y ordenar
train7 = train6.copy()
train7 = train7.sort_values(['prediction_unit_id', 'datetime'])

print(f"📊 train6 original: {len(train7):,} registros")

# Crear media móvil de target por prediction_unit_id (ventana 7 días, retraso 5 días)
train7['target_ma_7d_lag5d'] = (train7.groupby('prediction_unit_id')['target']
                               .shift(120)  # 5 días = 120 horas
                               .rolling(window=168, min_periods=168)  # 7 días = 168 horas
                               .mean())

# Stats de la nueva feature
valid_ma = train7['target_ma_7d_lag5d'].notna().sum()
print(f"📈 target_ma_7d_lag5d válidos: {valid_ma:,} ({valid_ma/len(train7)*100:.1f}%)")

# Eliminar columnas de historical_weather (no están en test set)
historical_weather_cols = [
    'temperature',      # histórico
    'dewpoint',         # histórico  
    'rain',
    'snowfall', 
    'surface_pressure',
    'cloudcover_low',
    'cloudcover_mid', 
    'cloudcover_high',
    'cloudcover_total',
    'windspeed_10m',
    'winddirection_10m', 
    'shortwave_radiation',
    'direct_solar_radiation',
    'diffuse_radiation'
]

# Verificar qué columnas existen y eliminarlas
cols_to_drop = [col for col in historical_weather_cols if col in train7.columns]
print(f"🗑️ Eliminando {len(cols_to_drop)} columnas weather históricas:")
for col in cols_to_drop:
    print(f"   - {col}")

train7 = train7.drop(columns=cols_to_drop)

print(f"✅ train7 creado: {len(train7):,} registros, {len(train7.columns)} columnas")

# Borrar train6 para liberar memoria
del train6
gc.collect()
print("🧹 train6 eliminado de memoria")


🔄 Creando train7: train6 + target_ma_7d_lag5d + sin weather histórico...
📊 train6 original: 7,821,024 registros
📈 target_ma_7d_lag5d válidos: 7,801,508 (99.8%)
🗑️ Eliminando 14 columnas weather históricas:
   - temperature
   - dewpoint
   - rain
   - snowfall
   - surface_pressure
   - cloudcover_low
   - cloudcover_mid
   - cloudcover_high
   - cloudcover_total
   - windspeed_10m
   - winddirection_10m
   - shortwave_radiation
   - direct_solar_radiation
   - diffuse_radiation
✅ train7 creado: 7,821,024 registros, 42 columnas
🧹 train6 eliminado de memoria


In [43]:
# # Modelo #26: Sin histórico weather (solo forecast weather)
# #✅ MAE sin weather histórico: 62.9796
# #📊 MAE con weather histórico: 61.2636
# # Eliminar columnas de historical_weather para ver impacto

# print("🔄 Modelo #26: Sin histórico weather...")

# # Filtrar últimos 90 días de train7
# max_date = train7['datetime'].max()
# cutoff_90d = max_date - pd.DateOffset(days=90)
# sample_90d = train7[train7['datetime'] >= cutoff_90d].copy()

# print(f"📊 Dataset 90 días: {len(sample_90d):,} registros")

# # Eliminar columnas de historical_weather (excepto datetime, latitude, longitude, data_block_id)
# historical_weather_cols = [
#     'temperature',      # histórico
#     'dewpoint',         # histórico  
#     'rain',
#     'snowfall', 
#     'surface_pressure',
#     'cloudcover_low',
#     'cloudcover_mid', 
#     'cloudcover_high',
#     'cloudcover_total',
#     'windspeed_10m',
#     'winddirection_10m', 
#     'shortwave_radiation',
#     'direct_solar_radiation',
#     'diffuse_radiation'
# ]

# # Verificar qué columnas existen y eliminarlas
# cols_to_drop = [col for col in historical_weather_cols if col in sample_90d.columns]
# print(f"🗑️ Eliminando {len(cols_to_drop)} columnas weather históricas:")
# for col in cols_to_drop:
#     print(f"   - {col}")

# sample_90d = sample_90d.drop(columns=cols_to_drop)

# # Limpiar NaN en media móvil
# sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
# print(f"📊 Datos finales: {len(sample_90d):,}")

# # Features (excluye target, datetime, forecast_date, row_id)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# print(f"🎯 Features totales: {len(feature_cols)} (sin weather histórico)")

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols]
# y_train = train_data['target']
# X_test = test_data[feature_cols]
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE sin weather histórico: {mae:.4f}")
# print(f"📊 MAE con weather histórico: 61.2636")


In [44]:
# # Modelo #27: Weather lags (variables solares) - 24h lag
# # ✅ MAE con weather lags 24h: 64.9544
# # 📊 MAE anterior: 62.9796


# print("🔄 Modelo #27: Weather lags 24h (variables solares)...")

# # Filtrar últimos 90 días de train7
# max_date = train7['datetime'].max()
# cutoff_90d = max_date - pd.DateOffset(days=90)
# sample_90d = train7[train7['datetime'] >= cutoff_90d].copy()

# print(f"📊 Dataset 90 días: {len(sample_90d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])

# # Variables solares para lag 24h
# solar_vars = ['ftemperature', 'fdirect_solar_radiation', 'fsurface_solar_radiation_downwards', 
#               'fcloudcover_total', 'fsnowfall']

# # Crear lags 24h de variables solares por prediction_unit_id
# for var in solar_vars:
#     if var in sample_90d.columns:
#         sample_90d[f'{var}_lag24h'] = sample_90d.groupby('prediction_unit_id')[var].shift(24)
        
#         # Stats del lag
#         valid_lags = sample_90d[f'{var}_lag24h'].notna().sum()
#         print(f"📈 {var}_lag24h válidos: {valid_lags:,}")

# # Limpiar NaN en media móvil (feature principal)
# sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
# print(f"📊 Datos finales: {len(sample_90d):,}")

# # Features (incluye nuevos weather lags)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# print(f"🎯 Features totales: {len(feature_cols)} (incluye weather lags)")

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols].fillna(0)  # Llenar NaN en lags con 0
# y_train = train_data['target']
# X_test = test_data[feature_cols].fillna(0)
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con weather lags 24h: {mae:.4f}")
# print(f"📊 MAE anterior: 62.9796")


In [45]:
# # Modelo #28: Weather lag 24h - solo ftemperature
# # ✅ MAE con ftemperature lag 24h: 63.2238
# # 📊 MAE anterior: 62.9796


# print("🔄 Modelo #28: Weather lag 24h - solo ftemperature...")

# # Filtrar últimos 90 días de train7
# max_date = train7['datetime'].max()
# cutoff_90d = max_date - pd.DateOffset(days=90)
# sample_90d = train7[train7['datetime'] >= cutoff_90d].copy()

# print(f"📊 Dataset 90 días: {len(sample_90d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])

# # Crear lag 24h solo de ftemperature
# if 'ftemperature' in sample_90d.columns:
#     sample_90d['ftemperature_lag24h'] = sample_90d.groupby('prediction_unit_id')['ftemperature'].shift(24)
    
#     # Stats del lag
#     valid_lags = sample_90d['ftemperature_lag24h'].notna().sum()
#     print(f"📈 ftemperature_lag24h válidos: {valid_lags:,}")
# else:
#     print("⚠️ ftemperature no encontrada")

# # Limpiar NaN en media móvil (feature principal)
# sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
# print(f"📊 Datos finales: {len(sample_90d):,}")

# # Features (incluye nuevo weather lag)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# print(f"🎯 Features totales: {len(feature_cols)} (incluye ftemperature lag)")

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols].fillna(0)  # Llenar NaN en lags con 0
# y_train = train_data['target']
# X_test = test_data[feature_cols].fillna(0)
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con ftemperature lag 24h: {mae:.4f}")
# print(f"📊 MAE anterior: 62.9796")


In [46]:
# # Modelo #29: Weather lag 48h - solo ftemperature
# # ✅ MAE con ftemperature lag 48h: 62.3245
# # 📊 MAE anterior: 62.9796


# print("🔄 Modelo #29: Weather lag 48h - solo ftemperature...")

# # Filtrar últimos 90 días de train7
# max_date = train7['datetime'].max()
# cutoff_90d = max_date - pd.DateOffset(days=90)
# sample_90d = train7[train7['datetime'] >= cutoff_90d].copy()

# print(f"📊 Dataset 90 días: {len(sample_90d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])

# # Crear lag 24h solo de ftemperature
# if 'ftemperature' in sample_90d.columns:
#     sample_90d['ftemperature_lag48h'] = sample_90d.groupby('prediction_unit_id')['ftemperature'].shift(48)
    
#     # Stats del lag
#     valid_lags = sample_90d['ftemperature_lag48h'].notna().sum()
#     print(f"📈 ftemperature_lag48h válidos: {valid_lags:,}")
# else:
#     print("⚠️ ftemperature no encontrada")

# # Limpiar NaN en media móvil (feature principal)
# sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
# print(f"📊 Datos finales: {len(sample_90d):,}")

# # Features (incluye nuevo weather lag)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# print(f"🎯 Features totales: {len(feature_cols)} (incluye ftemperature lag)")

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols].fillna(0)  # Llenar NaN en lags con 0
# y_train = train_data['target']
# X_test = test_data[feature_cols].fillna(0)
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con ftemperature lag 48h: {mae:.4f}")
# print(f"📊 MAE anterior: 62.9796")

In [47]:
# # Modelo 30
# # #🎉 Variables que MEJORAN (ordenadas por MAE):
# #    fdirect_solar_radiation_lag24h: 61.5811 (-1.3985)
# #    ftemperature_lag48h: 62.3245 (-0.6551)
# #    fcloudcover_total_lag48h: 62.3520 (-0.6276)
# def test_weather_lags():
#     """
#     Prueba cada variable weather con lag 24h y 48h individualmente
#     """
#     print("🔄 Creando todos los weather lags...")
    
#     # Variables solares a probar
#     solar_vars = ['ftemperature', 'fdirect_solar_radiation', 'fsurface_solar_radiation_downwards', 
#                   'fcloudcover_total', 'fsnowfall']
    
#     # Filtrar últimos 90 días de train7
#     max_date = train7['datetime'].max()
#     cutoff_90d = max_date - pd.DateOffset(days=90)
#     sample_90d = train7[train7['datetime'] >= cutoff_90d].copy()
    
#     # Ordenar por prediction_unit_id y datetime
#     sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])
    
#     # Crear TODOS los lags de una vez
#     print("📊 Creando lags...")
#     for var in solar_vars:
#         if var in sample_90d.columns:
#             # Lag 24h
#             sample_90d[f'{var}_lag24h'] = sample_90d.groupby('prediction_unit_id')[var].shift(24)
#             # Lag 48h  
#             sample_90d[f'{var}_lag48h'] = sample_90d.groupby('prediction_unit_id')[var].shift(48)
#             print(f"   ✅ {var} lags creados")
#         else:
#             print(f"   ⚠️ {var} no encontrada")
    
#     # Limpiar NaN en media móvil
#     sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
    
#     # Features base (sin los nuevos lags)
#     base_features = [col for col in sample_90d.columns 
#                      if col not in ['target', 'datetime', 'forecast_date', 'row_id'] 
#                      and not col.endswith('_lag24h') and not col.endswith('_lag48h')]
    
#     # Split 80/20 una sola vez
#     split_idx = int(len(sample_90d) * 0.8)
#     train_data = sample_90d.iloc[:split_idx]
#     test_data = sample_90d.iloc[split_idx:]
    
#     baseline_mae = 62.9796
#     results = []
    
#     print(f"\n🧪 Probando cada lag individualmente contra baseline {baseline_mae}...")
    
#     # Probar cada lag individualmente
#     for var in solar_vars:
#         if var in sample_90d.columns:
#             for lag_hours in [24, 48]:
#                 lag_col = f'{var}_lag{lag_hours}h'
                
#                 # Features = base + este lag específico
#                 test_features = base_features + [lag_col]
                
#                 X_train = train_data[test_features].fillna(0)
#                 y_train = train_data['target']
#                 X_test = test_data[test_features].fillna(0)
#                 y_test = test_data['target']
                
#                 # Modelo
#                 model = lgb.LGBMRegressor(
#                     n_estimators=100,
#                     max_depth=6,
#                     learning_rate=0.1,
#                     random_state=42,
#                     n_jobs=-1,
#                     verbose=-1
#                 )
                
#                 model.fit(X_train, y_train)
#                 pred = model.predict(X_test)
#                 mae = mean_absolute_error(y_test, pred)
                
#                 # A1: Resultado individual
#                 change = mae - baseline_mae
#                 status = "🎉 MEJORA" if mae < baseline_mae else "❌ EMPEORA"
                
#                 print(f"\n✅ MAE con {var} lag {lag_hours}h: {mae:.4f}")
#                 print(f"📊 Cambio: {change:+.4f} - {status}")
                
#                 # Guardar resultado
#                 results.append({
#                     'variable': var,
#                     'lag_hours': lag_hours,
#                     'mae': mae,
#                     'change': change,
#                     'improves': mae < baseline_mae
#                 })
    
#     # Resumen final
#     print("\n" + "="*60)
#     print("📊 RESUMEN DE RESULTADOS:")
#     print("="*60)
    
#     # Mejores resultados
#     improving = [r for r in results if r['improves']]
#     if improving:
#         improving.sort(key=lambda x: x['mae'])
#         print("\n🎉 Variables que MEJORAN (ordenadas por MAE):")
#         for r in improving:
#             print(f"   {r['variable']}_lag{r['lag_hours']}h: {r['mae']:.4f} ({r['change']:+.4f})")
    
#     # Peores resultados  
#     worsening = [r for r in results if not r['improves']]
#     if worsening:
#         worsening.sort(key=lambda x: x['change'], reverse=True)  # Menos empeoramiento primero
#         print(f"\n❌ Variables que EMPEORAN:")
#         for r in worsening:
#             print(f"   {r['variable']}_lag{r['lag_hours']}h: {r['mae']:.4f} ({r['change']:+.4f})")
    
#     return results

# # Ejecutar la función
# results = test_weather_lags()


# # #🎉 Variables que MEJORAN (ordenadas por MAE):
# #    fdirect_solar_radiation_lag24h: 61.5811 (-1.3985)
# #    ftemperature_lag48h: 62.3245 (-0.6551)
# #    fcloudcover_total_lag48h: 62.3520 (-0.6276)

# # ❌ Variables que EMPEORAN:
# #    fdirect_solar_radiation_lag48h: 67.6610 (+4.6814)
# #    fsurface_solar_radiation_downwards_lag48h: 65.9643 (+2.9847)
# #    fsnowfall_lag48h: 65.7171 (+2.7375)
# #    fsurface_solar_radiation_downwards_lag24h: 65.3518 (+2.3722)
# #    fcloudcover_total_lag24h: 64.2290 (+1.2494)
# #    fsnowfall_lag24h: 63.7094 (+0.7298)
# #    ftemperature_lag24h: 63.2238 (+0.2442)

In [48]:
# # Modelo #31: Weather lags (variables solares) 
# # fdirect_solar_radiation_lag24h: 61.5811 (-1.3985)
# # ftemperature_lag48h: 62.3245 (-0.6551)
# # fcloudcover_total_lag48h: 62.3520 (-0.6276)


# print("🔄 Modelo #31: Weather lags 24h (variables solares)...")

# # Filtrar últimos 90 días de train7
# max_date = train7['datetime'].max()
# cutoff_90d = max_date - pd.DateOffset(days=90)
# sample_90d = train7[train7['datetime'] >= cutoff_90d].copy()

# print(f"📊 Dataset 90 días: {len(sample_90d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])

# # Variables solares para lag 24h
# solar_vars24 = ['fdirect_solar_radiation']

# #Variables solares para lag 48h
# #solar_vars48 = ['ftemperature', 'fcloudcover_total']
# #solar_vars48 = ['ftemperature']
# #solar_vars48 = ['fcloudcover_total']

# #Crear lags 24h de variables solares por prediction_unit_id
# for var in solar_vars24:
#     if var in sample_90d.columns:
#         sample_90d[f'{var}_lag24h'] = sample_90d.groupby('prediction_unit_id')[var].shift(24)
        
#         # Stats del lag
#         valid_lags = sample_90d[f'{var}_lag24h'].notna().sum()
#         print(f"📈 {var}_lag24h válidos: {valid_lags:,}")

# # for var in solar_vars48:
# #     if var in sample_90d.columns:
# #         sample_90d[f'{var}_lag48h'] = sample_90d.groupby('prediction_unit_id')[var].shift(48)
        
# #         # Stats del lag
# #         valid_lags1 = sample_90d[f'{var}_lag48h'].notna().sum()
# #         print(f"📈 {var}_lag48h válidos: {valid_lags1:,}")

# # Limpiar NaN en media móvil (feature principal)
# sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
# print(f"📊 Datos finales: {len(sample_90d):,}")

# # Features (incluye nuevos weather lags)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# print(f"🎯 Features totales: {len(feature_cols)} (incluye weather lags)")

# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols].fillna(0)  # Llenar NaN en lags con 0
# y_train = train_data['target']
# X_test = test_data[feature_cols].fillna(0)
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE con weather lags 24 y 48h: {mae:.4f}")
# print(f"📊 MAE anterior: 62.9796")


In [49]:
# incorporo fdirect_solar_radiation_lag24h
train8 = train7.sort_values(['prediction_unit_id', 'datetime'])
train8['fdirect_solar_radiation_lag24h'] = train8.groupby('prediction_unit_id')['fdirect_solar_radiation'].shift(24)
print("✅ fdirect_solar_radiation_lag24h agregado a train8")


✅ fdirect_solar_radiation_lag24h agregado a train8


In [50]:
# # Modelo 36
# # 
# # fdewpoint_lag24h: 64.4357 (+2.8546) ❌ EMPEORA
# # fdewpoint_lag48h: 67.4056 (+5.8245) ❌ EMPEORA
# # fcloudcover_high_lag24h: 66.1002 (+4.5191) ❌ EMPEORA
# # fcloudcover_high_lag48h: 66.3328 (+4.7517) ❌ EMPEORA
# # fcloudcover_low_lag24h: 62.4589 (+0.8778) ❌ EMPEORA
# # fcloudcover_low_lag48h: 66.0350 (+4.4539) ❌ EMPEORA
# # fcloudcover_mid_lag24h: 64.7723 (+3.1912) ❌ EMPEORA
# # fcloudcover_mid_lag48h: 67.0185 (+5.4374) ❌ EMPEORA
# # f10_metre_u_wind_component_lag24h: 63.2744 (+1.6933) ❌ EMPEORA
# # f10_metre_u_wind_component_lag48h: 64.4923 (+2.9112) ❌ EMPEORA
# # f10_metre_v_wind_component_lag24h: 68.0118 (+6.4307) ❌ EMPEORA
# # f10_metre_v_wind_component_lag48h: 64.3495 (+2.7684) ❌ EMPEORA
# # ftotal_precipitation_lag24h: 64.6852 (+3.1041) ❌ EMPEORA
# # ftotal_precipitation_lag48h: 67.5245 (+5.9434) ❌ EMPEORA

# climate_vars = ['fdewpoint', 'fcloudcover_high', 'fcloudcover_low', 'fcloudcover_mid',
#                 'f10_metre_u_wind_component', 'f10_metre_v_wind_component', 'ftotal_precipitation']

# # Preparar datos
# max_date = train8['datetime'].max()
# sample_90d = train8[train8['datetime'] >= max_date - pd.DateOffset(days=90)].copy()
# sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])

# # Crear todos los lags climáticos
# for var in climate_vars:
#     sample_90d[f'{var}_lag24h'] = sample_90d.groupby('prediction_unit_id')[var].shift(24)
#     sample_90d[f'{var}_lag48h'] = sample_90d.groupby('prediction_unit_id')[var].shift(48)

# # Limpiar y split
# sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
# split_idx = int(len(sample_90d) * 0.8)
# train_data, test_data = sample_90d.iloc[:split_idx], sample_90d.iloc[split_idx:]

# # Features base
# base_features = [col for col in sample_90d.columns 
#                  if col not in ['target', 'datetime', 'forecast_date', 'row_id'] 
#                  and not any(f'{var}_lag' in col for var in climate_vars)]

# baseline_mae = 61.5811
# results = []

# # Probar cada lag individual
# for var in climate_vars:
#     for lag_hours in [24, 48]:
#         lag_col = f'{var}_lag{lag_hours}h'
#         test_features = base_features + [lag_col]
        
#         X_train = train_data[test_features].fillna(0)
#         X_test = test_data[test_features].fillna(0)
        
#         model = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, 
#                                  random_state=42, n_jobs=-1, verbose=-1)
#         model.fit(X_train, train_data['target'])
#         mae = mean_absolute_error(test_data['target'], model.predict(X_test))
        
#         change = mae - baseline_mae
#         status = "🎉 MEJORA" if mae < baseline_mae else "❌ EMPEORA"
#         print(f"{var}_lag{lag_hours}h: {mae:.4f} ({change:+.4f}) {status}")
        
#         results.append({'variable': var, 'lag_hours': lag_hours, 'mae': mae, 'change': change})

# # Mejores resultados
# improving = [r for r in results if r['mae'] < baseline_mae]
# if improving:
#     improving.sort(key=lambda x: x['mae'])
#     print("\n🎉 MEJORAN:")
#     for r in improving:
#         print(f"   {r['variable']}_lag{r['lag_hours']}h: {r['mae']:.4f} ({r['change']:+.4f})")
# else:
#     print("😔 Ninguna variable climática mejora")

In [51]:
# # Modelo #37
# # temp_solar_interaction: 63.5858 (+2.0047) ❌ EMPEORA
# # cloud_solar_ratio: 64.5474 (+2.9663) ❌ EMPEORA
# # wind_magnitude: 66.2764 (+4.6953) ❌ EMPEORA
# # temp_dewpoint_diff: 65.7131 (+4.1320) ❌ EMPEORA
# # surface_direct_ratio: 65.2653 (+3.6842) ❌ EMPEORA
# # ftemperature_roll3h: 64.2013 (+2.6202) ❌ EMPEORA
# # ftemperature_roll6h: 64.3444 (+2.7633) ❌ EMPEORA
# # fdirect_solar_radiation_roll3h: 64.2987 (+2.7176) ❌ EMPEORA
# # fdirect_solar_radiation_roll6h: 66.8305 (+5.2494) ❌ EMPEORA
# # fcloudcover_total_roll3h: 64.4380 (+2.8569) ❌ EMPEORA
# # fcloudcover_total_roll6h: 65.9477 (+4.3666) ❌ EMPEORA


# # Preparar datos base
# max_date = train8['datetime'].max()
# sample_90d = train8[train8['datetime'] >= max_date - pd.DateOffset(days=90)].copy()
# sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])

# # 1. INTERACCIONES WEATHER
# print("🔄 Creando interacciones weather...")
# sample_90d['temp_solar_interaction'] = sample_90d['ftemperature'] * sample_90d['fdirect_solar_radiation']
# sample_90d['cloud_solar_ratio'] = sample_90d['fdirect_solar_radiation'] / (sample_90d['fcloudcover_total'] + 0.01)
# sample_90d['wind_magnitude'] = np.sqrt(sample_90d['f10_metre_u_wind_component']**2 + sample_90d['f10_metre_v_wind_component']**2)

# # 2. DIFERENCIAS Y RATIOS
# sample_90d['temp_dewpoint_diff'] = sample_90d['ftemperature'] - sample_90d['fdewpoint']
# sample_90d['surface_direct_ratio'] = sample_90d['fsurface_solar_radiation_downwards'] / (sample_90d['fdirect_solar_radiation'] + 1)

# # 3. ROLLING FEATURES (ventanas más cortas)
# for var in ['ftemperature', 'fdirect_solar_radiation', 'fcloudcover_total']:
#     sample_90d[f'{var}_roll3h'] = sample_90d.groupby('prediction_unit_id')[var].rolling(3, min_periods=1).mean().values
#     sample_90d[f'{var}_roll6h'] = sample_90d.groupby('prediction_unit_id')[var].rolling(6, min_periods=1).mean().values

# # Limpiar y split
# sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
# split_idx = int(len(sample_90d) * 0.8)
# train_data, test_data = sample_90d.iloc[:split_idx], sample_90d.iloc[split_idx:]

# # Features base (con fdirect_solar_radiation_lag24h)
# base_features = [col for col in sample_90d.columns 
#                  if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Nuevas features a probar
# new_features = [
#     'temp_solar_interaction',
#     'cloud_solar_ratio', 
#     'wind_magnitude',
#     'temp_dewpoint_diff',
#     'surface_direct_ratio',
#     'ftemperature_roll3h',
#     'ftemperature_roll6h',
#     'fdirect_solar_radiation_roll3h',
#     'fdirect_solar_radiation_roll6h',
#     'fcloudcover_total_roll3h',
#     'fcloudcover_total_roll6h'
# ]

# # Features base sin las nuevas
# base_only = [col for col in base_features if col not in new_features]
# baseline_mae = 61.5811

# print(f"🧪 Probando {len(new_features)} nuevas features...")

# # Probar cada feature nueva individualmente
# results = []
# for feature in new_features:
#     if feature in sample_90d.columns:
#         test_features = base_only + [feature]
        
#         X_train = train_data[test_features].fillna(0)
#         X_test = test_data[test_features].fillna(0)
        
#         model = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, 
#                                  random_state=42, n_jobs=-1, verbose=-1)
#         model.fit(X_train, train_data['target'])
#         mae = mean_absolute_error(test_data['target'], model.predict(X_test))
        
#         change = mae - baseline_mae
#         status = "🎉 MEJORA" if mae < baseline_mae else "❌ EMPEORA"
#         print(f"{feature}: {mae:.4f} ({change:+.4f}) {status}")
        
#         results.append({'feature': feature, 'mae': mae, 'change': change})

# # Mejores resultados
# improving = [r for r in results if r['mae'] < baseline_mae]
# if improving:
#     improving.sort(key=lambda x: x['mae'])
#     print(f"\n🎉 MEJORAN ({len(improving)}):")
#     for r in improving:
#         print(f"   {r['feature']}: {r['mae']:.4f} ({r['change']:+.4f})")
# else:
#     # Mostrar las menos malas
#     results.sort(key=lambda x: x['change'])
#     print(f"\n🔝 TOP 3 menos malas:")
#     for r in results[:3]:
#         print(f"   {r['feature']}: {r['mae']:.4f} ({r['change']:+.4f})")

In [52]:
# # Modelo #38 la forma de calculo tiene filtracion de datos y aun asi no funciona
# # Clustering de prediction_units por comportamiento similar
# # cluster: 64.9886 (+3.4075) ❌ EMPEORA
# # cluster_hour_avg_target: 61.8005 (+0.2194) ❌ EMPEORA
# # 🔥 Probando ambas cluster features juntas...
# # cluster + cluster_hour_avg_target: 62.4489 (+0.8678) ❌ EMPEORA

# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler

# # 1. CREAR PERFIL DE CADA PREDICTION_UNIT
# print("📊 Creando perfiles de prediction_units...")

# # Usar train8 completo (150 días) para clustering
# # Agregaciones por prediction_unit_id para caracterizar comportamiento
# unit_profiles = train8.groupby('prediction_unit_id').agg({
#     'target': ['mean', 'std', 'min', 'max'],
#     'hour': lambda x: x.value_counts().idxmax(),  # hora más común
#     'is_consumption': 'mean',  # % tiempo consumiendo
#     'weekend': 'mean',  # % tiempo en weekend  
#     'installed_capacity': 'first',
#     'eic_count': 'first',
#     'is_business': 'first',
#     'county': 'first'
# }).round(4)

# # Aplanar nombres de columnas
# unit_profiles.columns = ['_'.join(col).strip() for col in unit_profiles.columns]
# unit_profiles = unit_profiles.reset_index()

# # 2. CLUSTERING
# print("🔄 Aplicando KMeans clustering...")

# # Features para clustering (solo numéricas)
# cluster_features = [
#     'target_mean', 'target_std', 'target_min', 'target_max',
#     'hour_<lambda>', 'is_consumption_mean', 'weekend_mean',
#     'installed_capacity_first', 'eic_count_first'
# ]

# # Preparar datos
# X_cluster = unit_profiles[cluster_features].fillna(0)
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_cluster)

# # Probar diferentes números de clusters
# silhouette_scores = []
# for n_clusters in range(3, 8):
#     kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
#     labels = kmeans.fit_predict(X_scaled)
    
#     from sklearn.metrics import silhouette_score
#     score = silhouette_score(X_scaled, labels)
#     silhouette_scores.append((n_clusters, score))
#     print(f"   K={n_clusters}: silhouette={score:.3f}")

# # Mejor número de clusters
# best_k = max(silhouette_scores, key=lambda x: x[1])[0]
# print(f"🎯 Mejor K: {best_k}")

# # Clustering final
# kmeans_final = KMeans(n_clusters=best_k, random_state=42, n_init=10)
# unit_profiles['cluster'] = kmeans_final.fit_predict(X_scaled)

# print(f"\n📈 Distribución de clusters:")
# print(unit_profiles['cluster'].value_counts().sort_index())

# # 3. AGREGAR CLUSTER AL DATASET ORIGINAL
# print("\n🔄 Agregando clusters a muestra de 90 días...")

# # Sample 90 días para testing rápido
# max_date = train8['datetime'].max()
# sample_90d = train8[train8['datetime'] >= max_date - pd.DateOffset(days=90)].copy()
# sample_90d = sample_90d.merge(
#     unit_profiles[['prediction_unit_id', 'cluster']], 
#     on='prediction_unit_id', 
#     how='left'
# )

# # Crear features adicionales basadas en cluster
# cluster_stats = sample_90d.groupby(['cluster', 'hour'])['target'].mean().reset_index()
# cluster_stats.columns = ['cluster', 'hour', 'cluster_hour_avg_target']

# sample_90d = sample_90d.merge(cluster_stats, on=['cluster', 'hour'], how='left')

# # 4. PROBAR CLUSTER FEATURES
# print("\n🧪 Probando cluster features...")

# sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
# split_idx = int(len(sample_90d) * 0.8)
# train_data, test_data = sample_90d.iloc[:split_idx], sample_90d.iloc[split_idx:]

# base_features = [col for col in sample_90d.columns 
#                  if col not in ['target', 'datetime', 'forecast_date', 'row_id', 'cluster', 'cluster_hour_avg_target']]

# baseline_mae = 61.5811

# # Probar features de cluster individualmente
# cluster_features_test = ['cluster', 'cluster_hour_avg_target']
# results = []

# for feature in cluster_features_test:
#     test_features = base_features + [feature]
    
#     X_train = train_data[test_features].fillna(0)
#     X_test = test_data[test_features].fillna(0)
    
#     model = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, 
#                              random_state=42, n_jobs=-1, verbose=-1)
#     model.fit(X_train, train_data['target'])
#     mae = mean_absolute_error(test_data['target'], model.predict(X_test))
    
#     change = mae - baseline_mae
#     status = "🎉 MEJORA" if mae < baseline_mae else "❌ EMPEORA"
#     print(f"{feature}: {mae:.4f} ({change:+.4f}) {status}")
    
#     results.append({'feature': feature, 'mae': mae, 'change': change})

# # Ambas features juntas
# print(f"\n🔥 Probando ambas cluster features juntas...")
# test_features = base_features + cluster_features_test

# X_train = train_data[test_features].fillna(0)
# X_test = test_data[test_features].fillna(0)

# model = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, 
#                          random_state=42, n_jobs=-1, verbose=-1)
# model.fit(X_train, train_data['target'])
# mae = mean_absolute_error(test_data['target'], model.predict(X_test))

# change = mae - baseline_mae
# status = "🎉 MEJORA" if mae < baseline_mae else "❌ EMPEORA"
# print(f"cluster + cluster_hour_avg_target: {mae:.4f} ({change:+.4f}) {status}")

# # Mostrar características de cada cluster
# print(f"\n📊 Perfil de clusters:")
# for cluster_id in sorted(unit_profiles['cluster'].unique()):
#     cluster_data = unit_profiles[unit_profiles['cluster'] == cluster_id]
#     n_units = len(cluster_data)
#     avg_target = cluster_data['target_mean_'].mean()
#     avg_consumption = cluster_data['is_consumption_mean'].mean()
    
#     print(f"   Cluster {cluster_id}: {n_units} units, target_avg={avg_target:.1f}, consumption%={avg_consumption:.1%}")

In [53]:
# #Modelo #39 long_lag_features = ['fdirect_solar_radiation_lag168h', 'fdirect_solar_radiation_lag336h', 'ftemperature_lag168h']
# # fdirect_solar_radiation_lag168h: 65.1855 (+3.6044) ❌ EMPEORA
# # fdirect_solar_radiation_lag336h: 65.7386 (+4.1575) ❌ EMPEORA
# # ftemperature_lag168h: 63.7261 (+2.1450) ❌ EMPEORA

# #LAGS MÁS LARGOS (evitar data leakage)
# print("🕐 Probando lags más largos de variables exitosas...")

# max_date = train8['datetime'].max()
# sample_90d = train8[train8['datetime'] >= max_date - pd.DateOffset(days=90)].copy()
# sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])

# # Lags de 7 días (168h) y 14 días (336h) de la variable que ya funciona
# sample_90d['fdirect_solar_radiation_lag168h'] = sample_90d.groupby('prediction_unit_id')['fdirect_solar_radiation'].shift(168)
# sample_90d['fdirect_solar_radiation_lag336h'] = sample_90d.groupby('prediction_unit_id')['fdirect_solar_radiation'].shift(336)

# # También probar otros lags de temperatura que casi funcionaba
# sample_90d['ftemperature_lag168h'] = sample_90d.groupby('prediction_unit_id')['ftemperature'].shift(168)

# sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
# split_idx = int(len(sample_90d) * 0.8)
# train_data, test_data = sample_90d.iloc[:split_idx], sample_90d.iloc[split_idx:]

# base_features = [col for col in sample_90d.columns 
#                  if col not in ['target', 'datetime', 'forecast_date', 'row_id'] 
#                  and not col.endswith('_lag168h') and not col.endswith('_lag336h')]

# baseline_mae = 61.5811
# long_lag_features = ['fdirect_solar_radiation_lag168h', 'fdirect_solar_radiation_lag336h', 'ftemperature_lag168h']

# for feature in long_lag_features:
#     test_features = base_features + [feature]
    
#     X_train = train_data[test_features].fillna(0)
#     X_test = test_data[test_features].fillna(0)
    
#     model = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, 
#                              random_state=42, n_jobs=-1, verbose=-1)
#     model.fit(X_train, train_data['target'])
#     mae = mean_absolute_error(test_data['target'], model.predict(X_test))
    
#     change = mae - baseline_mae
#     status = "🎉 MEJORA" if mae < baseline_mae else "❌ EMPEORA"
#     print(f"{feature}: {mae:.4f} ({change:+.4f}) {status}")

# print("\n" + "="*50)

# print(f"\n💰 RESUMEN: Actualmente tienes MAE {baseline_mae:.4f}")


In [54]:
# # Modelo #40 fdirect_solar_radiation la problematica , train8 tiene 150dias y luego corta a 90dias train8 tiene incorporada la variable
# #                                                      train7 corta primero a 90dias y luego calcula , incorpora aca la variable
# #✅ MAE: 64.9886 con train8
# # MAE: 62.9796 con train7

# # Filtrar últimos 90 días de train8
# max_date = train8['datetime'].max()
# cutoff_90d = max_date - pd.DateOffset(days=90)
# sample_90d = train8[train8['datetime'] >= cutoff_90d].copy()

# print(f"📊 Dataset 90 días: {len(sample_90d):,} registros")

# # Ordenar por prediction_unit_id y datetime
# sample_90d = sample_90d.sort_values(['prediction_unit_id', 'datetime'])

# # Limpiar NaN en media móvil (feature principal)
# sample_90d = sample_90d.dropna(subset=['target_ma_7d_lag5d'])
# print(f"📊 Datos finales: {len(sample_90d):,}")

# # Features (incluye nuevos weather lags)
# feature_cols = [col for col in sample_90d.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]


# # Split 80/20
# split_idx = int(len(sample_90d) * 0.8)
# train_data = sample_90d.iloc[:split_idx]
# test_data = sample_90d.iloc[split_idx:]

# X_train = train_data[feature_cols].fillna(0)  # Llenar NaN en lags con 0
# y_train = train_data['target']
# X_test = test_data[feature_cols].fillna(0)
# y_test = test_data['target']

# # Modelo LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultado
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"\n✅ MAE: {mae:.4f}")


In [55]:
# Modelo #41 pruebas con distintios dias ,parace haber concept drift

# Días	Train7 MAE	Train8 MAE	Diferencia	Ganador
# -------------------------------------------------------
# 90	61.5811		64.9886		+3.4075		TRAIN7 #modelo 31
# 100	63.9248		63.4369		-0.4878		TRAIN8
# 110	64.4976		65.3186		+0.8210		TRAIN7
# 120	65.7664		66.9320		+1.1656		TRAIN7

# def test_single_cutoff_train7(days):
#     """Probar UN cutoff con método train7"""
#     print(f"\n🔄 TRAIN7 - {days} días...")
    
#     max_date = train7['datetime'].max()
#     cutoff = max_date - pd.DateOffset(days=days)
    
#     # Cortar primero
#     sample = train7[train7['datetime'] >= cutoff].copy()
    
#     # Agregar lag después
#     sample = sample.sort_values(['prediction_unit_id', 'datetime'])
#     sample['fdirect_solar_radiation_lag24h'] = sample.groupby('prediction_unit_id')['fdirect_solar_radiation'].shift(24)
    
#     # Limpiar
#     sample = sample.dropna(subset=['target_ma_7d_lag5d'])
#     feature_cols = [col for col in sample.columns 
#                    if col not in ['target', 'datetime', 'forecast_date', 'row_id']]
    
#     # Stats
#     valid_lags = sample['fdirect_solar_radiation_lag24h'].notna().sum()
#     total_records = len(sample)
    
#     # Split y modelo
#     split_idx = int(len(sample) * 0.8)
#     train_data = sample.iloc[:split_idx]
#     test_data = sample.iloc[split_idx:]
    
#     X_train = train_data[feature_cols].fillna(0)
#     y_train = train_data['target']
#     X_test = test_data[feature_cols].fillna(0)
#     y_test = test_data['target']
    
#     # Limpiar muestras grandes
#     del sample, train_data, test_data
#     gc.collect()
    
#     # Modelo
#     model = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1,
#                              random_state=42, n_jobs=-1, verbose=-1)
#     model.fit(X_train, y_train)
#     mae = mean_absolute_error(y_test, model.predict(X_test))
    
#     # Limpiar modelo
#     del model, X_train, X_test, y_train, y_test
#     gc.collect()
    
#     print(f"   📈 Registros: {total_records:,} | Lags válidos: {valid_lags:,} | MAE: {mae:.4f}")
#     return {'days': days, 'mae': mae, 'records': total_records, 'valid_lags': valid_lags}

# def test_single_cutoff_train8(days):
#     """Probar UN cutoff con método train8"""
#     print(f"\n🔄 TRAIN8 - {days} días...")
    
#     max_date = train8['datetime'].max()
#     cutoff = max_date - pd.DateOffset(days=days)
    
#     # Cortar (lag ya existe)
#     sample = train8[train8['datetime'] >= cutoff].copy()
#     sample = sample.sort_values(['prediction_unit_id', 'datetime'])
#     sample = sample.dropna(subset=['target_ma_7d_lag5d'])
    
#     feature_cols = [col for col in sample.columns 
#                    if col not in ['target', 'datetime', 'forecast_date', 'row_id']]
    
#     # Stats
#     valid_lags = sample['fdirect_solar_radiation_lag24h'].notna().sum()
#     total_records = len(sample)
    
#     # Split y modelo
#     split_idx = int(len(sample) * 0.8)
#     train_data = sample.iloc[:split_idx]
#     test_data = sample.iloc[split_idx:]
    
#     X_train = train_data[feature_cols].fillna(0)
#     y_train = train_data['target']
#     X_test = test_data[feature_cols].fillna(0)
#     y_test = test_data['target']
    
#     # Limpiar muestras grandes
#     del sample, train_data, test_data
#     gc.collect()
    
#     # Modelo
#     model = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1,
#                              random_state=42, n_jobs=-1, verbose=-1)
#     model.fit(X_train, y_train)
#     mae = mean_absolute_error(y_test, model.predict(X_test))
    
#     # Limpiar modelo
#     del model, X_train, X_test, y_train, y_test
#     gc.collect()
    
#     print(f"   📈 Registros: {total_records:,} | Lags válidos: {valid_lags:,} | MAE: {mae:.4f}")
#     return {'days': days, 'mae': mae, 'records': total_records, 'valid_lags': valid_lags}

# # EJECUTAR DE A UNO
# print("🚀 Probando cutoffs de a uno...")
# results = []

# cutoff_days = [90, 100, 110, 120]

# for days in cutoff_days:
#     print(f"\n{'='*50}")
#     print(f"PROBANDO {days} DÍAS")
#     print(f"{'='*50}")
    
#     # Train7
#     result_t7 = test_single_cutoff_train7(days)
    
#     # Train8  
#     result_t8 = test_single_cutoff_train8(days)
    
#     # Comparar
#     diff = result_t8['mae'] - result_t7['mae']
#     winner = "TRAIN7" if result_t7['mae'] < result_t8['mae'] else "TRAIN8"
    
#     print(f"\n🎯 RESUMEN {days} días:")
#     print(f"   Train7: {result_t7['mae']:.4f}")
#     print(f"   Train8: {result_t8['mae']:.4f}")
#     print(f"   Diferencia: {diff:+.4f}")
#     print(f"   Ganador: {winner}")
    
#     results.append({
#         'days': days,
#         'train7_mae': result_t7['mae'],
#         'train8_mae': result_t8['mae'],
#         'difference': diff,
#         'winner': winner
#     })
    
#     # Forzar limpieza memoria
#     gc.collect()

# # RESUMEN FINAL
# print(f"\n{'='*60}")
# print("📊 RESUMEN FINAL")
# print(f"{'='*60}")

# print("\nDías\tTrain7 MAE\tTrain8 MAE\tDiferencia\tGanador")
# print("-" * 55)
# for r in results:
#     print(f"{r['days']}\t{r['train7_mae']:.4f}\t\t{r['train8_mae']:.4f}\t\t{r['difference']:+.4f}\t\t{r['winner']}")

# # Mejor configuración general
# best_overall = min(results, key=lambda x: min(x['train7_mae'], x['train8_mae']))
# best_mae = min(best_overall['train7_mae'], best_overall['train8_mae'])
# best_method = "Train7" if best_overall['train7_mae'] < best_overall['train8_mae'] else "Train8"

# print(f"\n🏆 MEJOR CONFIGURACIÓN GENERAL:")
# print(f"   {best_method} con {best_overall['days']} días → MAE {best_mae:.4f}")

# # Contar victorias
# train7_wins = sum(1 for r in results if r['winner'] == 'TRAIN7')
# train8_wins = sum(1 for r in results if r['winner'] == 'TRAIN8')

# print(f"\n📊 ESTADÍSTICAS:")
# print(f"   Train7 ganó: {train7_wins}/4 veces")
# print(f"   Train8 ganó: {train8_wins}/4 veces")

# if train7_wins > train8_wins:
#     print("🎯 CONCLUSIÓN: Método Train7 (corta → lag) es mejor")
# elif train8_wins > train7_wins:
#     print("🎯 CONCLUSIÓN: Método Train8 (lag → corta) es mejor") 
# else:
#     print("🎯 CONCLUSIÓN: Ambos métodos son equivalentes")

🚀 Probando cutoffs de a uno...

PROBANDO 90 DÍAS

🔄 TRAIN7 - 90 días...
   📈 Registros: 4,622,184 | Lags válidos: 4,620,552 | MAE: 61.5811

🔄 TRAIN8 - 90 días...
   📈 Registros: 4,622,184 | Lags válidos: 4,622,184 | MAE: 64.9886

🎯 RESUMEN 90 días:
   Train7: 61.5811
   Train8: 64.9886
   Diferencia: +3.4075
   Ganador: TRAIN7

PROBANDO 100 DÍAS

🔄 TRAIN7 - 100 días...
   📈 Registros: 5,150,664 | Lags válidos: 5,149,032 | MAE: 63.9248

🔄 TRAIN8 - 100 días...
   📈 Registros: 5,150,664 | Lags válidos: 5,150,664 | MAE: 63.4369

🎯 RESUMEN 100 días:
   Train7: 63.9248
   Train8: 63.4369
   Diferencia: -0.4878
   Ganador: TRAIN8

PROBANDO 110 DÍAS

🔄 TRAIN7 - 110 días...
   📈 Registros: 5,682,913 | Lags válidos: 5,681,305 | MAE: 64.4976

🔄 TRAIN8 - 110 días...
   📈 Registros: 5,682,913 | Lags válidos: 5,682,913 | MAE: 65.3186

🎯 RESUMEN 110 días:
   Train7: 64.4976
   Train8: 65.3186
   Diferencia: +0.8210
   Ganador: TRAIN7

PROBANDO 120 DÍAS

🔄 TRAIN7 - 120 días...
   📈 Registros: 6,219,52

In [61]:
# Modelo 42
# Días	Train7 MAE	Train8 MAE	Diferencia	Ganador
# -------------------------------------------------------
# 60	62.6199		63.9901		+1.3702		TRAIN7
# 70	66.6343		62.8047		-3.8296		TRAIN8
# 80	62.7194		63.1605		+0.4411		TRAIN7
# 90	61.5811		64.9886		+3.4075		TRAIN7

def test_single_cutoff_train7(days):
    """Probar UN cutoff con método train7"""
    print(f"\n🔄 TRAIN7 - {days} días...")
    
    max_date = train7['datetime'].max()
    cutoff = max_date - pd.DateOffset(days=days)
    
    # Cortar primero
    sample = train7[train7['datetime'] >= cutoff].copy()
    
    # Agregar lag después
    sample = sample.sort_values(['prediction_unit_id', 'datetime'])
    sample['fdirect_solar_radiation_lag24h'] = sample.groupby('prediction_unit_id')['fdirect_solar_radiation'].shift(24)
    
    # Limpiar
    sample = sample.dropna(subset=['target_ma_7d_lag5d'])
    feature_cols = [col for col in sample.columns 
                   if col not in ['target', 'datetime', 'forecast_date', 'row_id']]
    
    # Stats
    valid_lags = sample['fdirect_solar_radiation_lag24h'].notna().sum()
    total_records = len(sample)
    
    # Split y modelo
    split_idx = int(len(sample) * 0.8)
    train_data = sample.iloc[:split_idx]
    test_data = sample.iloc[split_idx:]
    
    X_train = train_data[feature_cols].fillna(0)
    y_train = train_data['target']
    X_test = test_data[feature_cols].fillna(0)
    y_test = test_data['target']
    
    # Limpiar muestras grandes
    del sample, train_data, test_data
    gc.collect()
    
    # Modelo
    model = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1,
                             random_state=42, n_jobs=-1, verbose=-1)
    model.fit(X_train, y_train)
    mae = mean_absolute_error(y_test, model.predict(X_test))
    
    # Limpiar modelo
    del model, X_train, X_test, y_train, y_test
    gc.collect()
    
    print(f"   📈 Registros: {total_records:,} | Lags válidos: {valid_lags:,} | MAE: {mae:.4f}")
    return {'days': days, 'mae': mae, 'records': total_records, 'valid_lags': valid_lags}

def test_single_cutoff_train8(days):
    """Probar UN cutoff con método train8"""
    print(f"\n🔄 TRAIN8 - {days} días...")
    
    max_date = train8['datetime'].max()
    cutoff = max_date - pd.DateOffset(days=days)
    
    # Cortar (lag ya existe)
    sample = train8[train8['datetime'] >= cutoff].copy()
    sample = sample.sort_values(['prediction_unit_id', 'datetime'])
    sample = sample.dropna(subset=['target_ma_7d_lag5d'])
    
    feature_cols = [col for col in sample.columns 
                   if col not in ['target', 'datetime', 'forecast_date', 'row_id']]
    
    # Stats
    valid_lags = sample['fdirect_solar_radiation_lag24h'].notna().sum()
    total_records = len(sample)
    
    # Split y modelo
    split_idx = int(len(sample) * 0.8)
    train_data = sample.iloc[:split_idx]
    test_data = sample.iloc[split_idx:]
    
    X_train = train_data[feature_cols].fillna(0)
    y_train = train_data['target']
    X_test = test_data[feature_cols].fillna(0)
    y_test = test_data['target']
    
    # Limpiar muestras grandes
    del sample, train_data, test_data
    gc.collect()
    
    # Modelo
    model = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1,
                             random_state=42, n_jobs=-1, verbose=-1)
    model.fit(X_train, y_train)
    mae = mean_absolute_error(y_test, model.predict(X_test))
    
    # Limpiar modelo
    del model, X_train, X_test, y_train, y_test
    gc.collect()
    
    print(f"   📈 Registros: {total_records:,} | Lags válidos: {valid_lags:,} | MAE: {mae:.4f}")
    return {'days': days, 'mae': mae, 'records': total_records, 'valid_lags': valid_lags}

# EJECUTAR DE A UNO
print("🚀 Probando cutoffs de a uno...")
results = []

cutoff_days = [60, 70, 80, 90]

for days in cutoff_days:
    print(f"\n{'='*50}")
    print(f"PROBANDO {days} DÍAS")
    print(f"{'='*50}")
    
    # Train7
    result_t7 = test_single_cutoff_train7(days)
    
    # Train8  
    result_t8 = test_single_cutoff_train8(days)
    
    # Comparar
    diff = result_t8['mae'] - result_t7['mae']
    winner = "TRAIN7" if result_t7['mae'] < result_t8['mae'] else "TRAIN8"
    
    print(f"\n🎯 RESUMEN {days} días:")
    print(f"   Train7: {result_t7['mae']:.4f}")
    print(f"   Train8: {result_t8['mae']:.4f}")
    print(f"   Diferencia: {diff:+.4f}")
    print(f"   Ganador: {winner}")
    
    results.append({
        'days': days,
        'train7_mae': result_t7['mae'],
        'train8_mae': result_t8['mae'],
        'difference': diff,
        'winner': winner
    })
    
    # Forzar limpieza memoria
    gc.collect()

# RESUMEN FINAL
print(f"\n{'='*60}")
print("📊 RESUMEN FINAL")
print(f"{'='*60}")

print("\nDías\tTrain7 MAE\tTrain8 MAE\tDiferencia\tGanador")
print("-" * 55)
for r in results:
    print(f"{r['days']}\t{r['train7_mae']:.4f}\t\t{r['train8_mae']:.4f}\t\t{r['difference']:+.4f}\t\t{r['winner']}")

# Mejor configuración general
best_overall = min(results, key=lambda x: min(x['train7_mae'], x['train8_mae']))
best_mae = min(best_overall['train7_mae'], best_overall['train8_mae'])
best_method = "Train7" if best_overall['train7_mae'] < best_overall['train8_mae'] else "Train8"

print(f"\n🏆 MEJOR CONFIGURACIÓN GENERAL:")
print(f"   {best_method} con {best_overall['days']} días → MAE {best_mae:.4f}")

# Contar victorias
train7_wins = sum(1 for r in results if r['winner'] == 'TRAIN7')
train8_wins = sum(1 for r in results if r['winner'] == 'TRAIN8')

print(f"\n📊 ESTADÍSTICAS:")
print(f"   Train7 ganó: {train7_wins}/4 veces")
print(f"   Train8 ganó: {train8_wins}/4 veces")

if train7_wins > train8_wins:
    print("🎯 CONCLUSIÓN: Método Train7 (corta → lag) es mejor")
elif train8_wins > train7_wins:
    print("🎯 CONCLUSIÓN: Método Train8 (lag → corta) es mejor") 
else:
    print("🎯 CONCLUSIÓN: Ambos métodos son equivalentes")

🚀 Probando cutoffs de a uno...

PROBANDO 60 DÍAS

🔄 TRAIN7 - 60 días...
   📈 Registros: 3,069,912 | Lags válidos: 3,068,304 | MAE: 62.6199

🔄 TRAIN8 - 60 días...
   📈 Registros: 3,069,912 | Lags válidos: 3,069,912 | MAE: 63.9901

🎯 RESUMEN 60 días:
   Train7: 62.6199
   Train8: 63.9901
   Diferencia: +1.3702
   Ganador: TRAIN7

PROBANDO 70 DÍAS

🔄 TRAIN7 - 70 días...
   📈 Registros: 3,585,384 | Lags válidos: 3,583,752 | MAE: 66.6343

🔄 TRAIN8 - 70 días...
   📈 Registros: 3,585,384 | Lags válidos: 3,585,384 | MAE: 62.8047

🎯 RESUMEN 70 días:
   Train7: 66.6343
   Train8: 62.8047
   Diferencia: -3.8296
   Ganador: TRAIN8

PROBANDO 80 DÍAS

🔄 TRAIN7 - 80 días...
   📈 Registros: 4,103,784 | Lags válidos: 4,102,152 | MAE: 62.7194

🔄 TRAIN8 - 80 días...
   📈 Registros: 4,103,784 | Lags válidos: 4,103,784 | MAE: 63.1605

🎯 RESUMEN 80 días:
   Train7: 62.7194
   Train8: 63.1605
   Diferencia: +0.4411
   Ganador: TRAIN7

PROBANDO 90 DÍAS

🔄 TRAIN7 - 90 días...
   📈 Registros: 4,622,184 | Lags v

In [56]:
train7.head()

,row_id,target,datetime,forecast_date,hour,data_block_id,prediction_unit_id,is_business,product_type,county,latitude,longitude,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity,ftemperature,fdewpoint,fcloudcover_high,fcloudcover_low,fcloudcover_mid,fcloudcover_total,f10_metre_u_wind_component,f10_metre_v_wind_component,fdirect_solar_radiation,fsurface_solar_radiation_downwards,fsnowfall,ftotal_precipitation,weather_forecast_hour,is_consumption,day_of_week,month,is_daylight,weekend,capacity_per_obs,production_obs_ratio,consumption_obs_ratio,capacity_per_capita,electricity_price_volatility_7d,gas_price_volatility_7d,target_ma_7d_lag5d
0,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.099998,24.200001,74.0,86.199997,3.29,417.0,4001.080078,2.458154,1.860742,0.989075,1.0,1.000000,0.999996,0.320052,8.630277,-0.008889,5.534878,0.000023,0.000508,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0,NaN
1,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.099998,25.200001,74.0,86.199997,3.29,417.0,4001.080078,1.464502,0.857690,0.897797,1.0,1.000000,0.999996,0.415511,7.309232,0.008889,15.983768,0.000063,0.000489,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0,NaN
2,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.400002,22.700001,74.0,86.199997,3.29,417.0,4001.080078,5.330591,3.928491,0.064026,1.0,0.054596,0.999996,4.160628,12.238919,0.008889,5.841545,0.000000,0.000222,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0,NaN
3,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.400002,23.200001,74.0,86.199997,3.29,417.0,4001.080078,4.218042,2.536279,0.132874,1.0,0.371002,0.999996,1.823714,13.567777,0.000000,5.081545,0.000000,0.000197,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0,NaN
4,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.400002,23.700001,74.0,86.199997,3.29,417.0,4001.080078,3.586084,2.155054,0.618713,1.0,1.000000,0.999996,0.730208,12.648099,0.000000,3.925990,0.000000,0.000252,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0,NaN


In [57]:
train8.head()

,row_id,target,datetime,forecast_date,hour,data_block_id,prediction_unit_id,is_business,product_type,county,latitude,longitude,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity,ftemperature,fdewpoint,fcloudcover_high,fcloudcover_low,fcloudcover_mid,fcloudcover_total,f10_metre_u_wind_component,f10_metre_v_wind_component,fdirect_solar_radiation,fsurface_solar_radiation_downwards,fsnowfall,ftotal_precipitation,weather_forecast_hour,is_consumption,day_of_week,month,is_daylight,weekend,capacity_per_obs,production_obs_ratio,consumption_obs_ratio,capacity_per_capita,electricity_price_volatility_7d,gas_price_volatility_7d,target_ma_7d_lag5d,fdirect_solar_radiation_lag24h
0,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.099998,24.200001,74.0,86.199997,3.29,417.0,4001.080078,2.458154,1.860742,0.989075,1.0,1.000000,0.999996,0.320052,8.630277,-0.008889,5.534878,0.000023,0.000508,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0,NaN,NaN
1,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.099998,25.200001,74.0,86.199997,3.29,417.0,4001.080078,1.464502,0.857690,0.897797,1.0,1.000000,0.999996,0.415511,7.309232,0.008889,15.983768,0.000063,0.000489,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0,NaN,NaN
2,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.400002,22.700001,74.0,86.199997,3.29,417.0,4001.080078,5.330591,3.928491,0.064026,1.0,0.054596,0.999996,4.160628,12.238919,0.008889,5.841545,0.000000,0.000222,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0,NaN,NaN
3,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.400002,23.200001,74.0,86.199997,3.29,417.0,4001.080078,4.218042,2.536279,0.132874,1.0,0.371002,0.999996,1.823714,13.567777,0.000000,5.081545,0.000000,0.000197,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0,NaN,NaN
4,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.400002,23.700001,74.0,86.199997,3.29,417.0,4001.080078,3.586084,2.155054,0.618713,1.0,1.000000,0.999996,0.730208,12.648099,0.000000,3.925990,0.000000,0.000252,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0,NaN,NaN


In [58]:
#Mae puesto n1 kaggle 52.3090
#1er primer modelo xgb sobre train5 haciendo split 80/20 de train y test 
#Mae = 107.9896
#sin lag sin feature, sin nada , solo haciendo el merge


#agrego day of week y month para todos los siguientes

#2do modelo lgb 10% y sobre esto split 80/20 de train y test 
#Mae = 89.3

#3er modelo lgb 10% y sobre esto split 80/20 de train y test 
#Mae = 103.1249
#climate_vars = ['fsurface_solar_radiation_downwards', 'shortwave_radiation', 
#                'temperature', 'direct_solar_radiation']
# agrego lags de 1h/4h/12h en las variables de arriba
# empeoro porque agregue los lags al azar no hice un analisis para agregarlo en los marcos de tiempo relevante
# estas variables de lags las quito

#agrego is_daylight binaria

#4to modelo lgb 14.2% y sobre esto split 80/20 de train y test  
#de base solo agrego la variable is_daylight , pero ahora uso info de los ultimos 3 meses 14.2% aprox
# se que no deberia ser comparable al usar de base distinto , pero parte de aca ahora
#Mae = 86.6747

#agrego weekend binaria

#5to modelo lgb igual que el anterior , solo agregue binaria Weekend y no vi cambios
#Mae = 86.6747

#para todos los demas modelos ya tienen incluido
#day of week
#month
#is_daylight
#weekend

#6to modelo lgb , agregue variable binaria is_active_hours , horario de 8 a 17 hs binaria
#Mae = 90.4776 , empeoro el modelo la saco por ahora a is_active_hours

#7mo modelo lgb, agregue variable binaria is_sleeping_hours 22h-6h
#Mae = 98.3152 empeora el modelo , asi que  la saco a is_sleeping_hours

#empiezo a probar con variables
##Energéticas:

# Ratio production/consumption por prediction_unit_id (histórico) # no se pudo probar
# Capacidad instalada per capita (installed_capacity / eic_count) # si mejora el modelo
# Eficiencia de paneles (production vs solar_radiation cuando is_consumption=False)
# Volatilidad de precios (rolling std de electricity/gas prices)
# Spread gas-electricidad (diferencia de precios, importante para switching)

#8vo modelo lgb agregue 3 variables energeticas capacity_per_obs, production_obs_ratio, consumption_obs_ratio
#Mae = 76.2703 mejor bastante el modelo

#9no modelo lgb  mantengo las 3 anteriores y prueba una nueva Ratio production/cn
# no optimizado no pudo correr

#10mo modelo lgb mantengo las 3 anteriores y pruebo  Capacidad instalada per capita (installed_capacity / eic_count)
#Mae = 72.3572 mejora el modelo 

#para los demas modelos sigo usando las siguientes variables
#capacity_per_obs
#production_obs_ratio
#consumption_obs_ratio
#capacity_per_capita

#11vo modelo lgb con esto #Eficiencia de paneles (production vs solar_radiation cuando is_consumption=False)#
#Mae anterior = 72.3572
#Mae con eficiencia de paneles = 85.0309 empeora , pruebo con lag de esto  y sino no la uso por ahora


#12vo modelo lgb , lo anterior pero ahora efficiency panel es log , lo saco
# ✅ MAE con panel efficiency LOG: 85.3200
# 📊 MAE anterior: 72.3572


# #13vo modelo probando volatilidad de precio de electricidad 7 dias
# # ✅ MAE con electricity price volatility: 73.2246 si , falta probar mas versiones y gas
# # 📊 MAE anterior: 72.3572

#14vo modelo ademas de electricity_volatility pruebo con gas
# ✅ MAE con electricity + gas price volatility: 70.6517 se queda
# 📊 MAE anterior: 73.2246

#creo train6 de 150 dias para agregar
#capacity_per_obs
#production_obs_ratio
#consumption_obs_ratio
#capacity_per_capita
#elect price volatility
#gas price volatility

#15vo probar gas y/o electricity de 30 dias , agrega mucho ruido no sirve

# 16vo modelo Lag 24h de target por prediction_unit_id , no va
#✅ MAE con lag 24h (90 días): 104.3003
#📊 MAE anterior: 70.6517

# 17vo modelo Lag 48h de target por prediction_unit_id , no va
#✅ MAE con lag 48h (90 días): 111.3501
#📊 MAE anterior: 70.6517

# # Modelo #18: Cyclical encoding de hora (hour_sin, hour_cos)
# ✅ MAE con hour_sin/cos: 122.4835 # algo esta pasado ver como corregirlo
# 📊 MAE anterior: 70.6517

# Modelo #19: Cyclical encoding de semana (week_sin, week_cos)
#✅ MAE con week_sin/cos: 118.3710 los temporales , de la forma que los planteo no estan funcionando
#📊 MAE anterior: 70.6517

# Modelo #20: Cambios día a día (delta vs día anterior) 'temperature', 'euros_per_mwh', 'lowest_price_per_mwh', 'direct_solar_radiation'
# ✅ MAE con deltas weather 24h: 124.1003 algo mal estoy haciendo o el overfit es grande por 90dias ,capaz con mas dias menos overfit
# 📊 MAE anterior: 70.6517

# Modelo #21: Media móvil de target por prediction_unit_id
# ✅ MAE con media móvil 7d: 56.1320 MMM data leakage por como estan los datos , veo como lo cambio
# 📊 MAE anterior: 70.6517

# Modelo #22: Media móvil de target por prediction_unit_id lag 5 dias
# ✅ MAE con media móvil 7d (lag 5d): 61.2636
# 📊 MAE anterior: 70.6517

# Modelo #23: Media móvil de target por prediction_unit_id lag 5 dias  
# voy a probar con una sola de las 2 y tambien probar de 5 dias para ser consistente con el lag
#✅ MAE con ambas media móvil 7d y 3d (lag 5d): 64.0943
#📊 MAE anterior: 61.2636

 
# Modelo #24: Media móvil de target por prediction_unit_id lag 5 dias
#✅ MAE con media móvil 3d (lag 5d): 63.5897 # sigue siendo mejor solo la de 7 dias
#📊 MAE anterior: 61.2636

# Modelo #25: Media móvil de target por prediction_unit_id lag 5 dias
#✅ MAE con media móvil 5d (lag 5d): 71.2301 # sigue siendo mejor solo la de 7 dias
#📊 MAE anterior: 61.2636

#creo train7 que contiene la media movil de 7d con lag de 5d

# Modelo #26 (deberia haber sido antes) forecast Weather vs historical Weather (historical no esta en test)
# teniendo en cuenta varias cosas esto deberia haber sido hecho antes , tanto por memoria/optimizacion/forma prolijas de trabajar y etc
# mismo el pipeline deberia haber sido hecho solo con info de los ultimos 6 meses para entrenar mas rapido y no cortarlo despues
# pese a los varios e importantes errores que tuve sigo probando unas cosas mas en este dataset
# agrego esto a cuando creo train7 y sigo ese como base
#✅ MAE sin weather histórico: 62.9796
#📊 MAE con weather histórico: 61.2636

# Modelo #27: Weather lags (variables solares) - 24h lag
# ✅ MAE con weather lags 24h: 64.9544 #prueba de 5 a la vez , no se si alguna reduce y las otras son ruido , paso a probar de a 1
# 📊 MAE anterior: 62.9796

# Modelo #28: Weather lag 24h - solo ftemperature
# ✅ MAE con ftemperature lag 24h: 63.2238 # nop , pruebo 48h y sigo con la siguiente
# 📊 MAE anterior: 62.9796

# # Modelo #29: Weather lag 48h - solo ftemperature
# # ✅ MAE con ftemperature lag 48h: 62.3245 #nop , pruebo los demas , igual tendria que hacer una funcion que haga esto con los otras variables 
# #  y en 24h y 48h
# # 📊 MAE anterior: 62.9796

# Modelo #30
# #🎉 Variables que MEJORAN (ordenadas por MAE):
#    fdirect_solar_radiation_lag24h: 61.5811 (-1.3985)
#     ftemperature_lag48h y: 62.3245 (-0.6551)
#    fcloudcover_total_lag48h: 62.3520 (-0.6276)

#la verdad es que deberia automatizar esto como #mlxtend.feature_selection.SequentialFeatureSelector
#cuando vaya a probar las otras variables que no sean solares lo pruebo

# Modelo #31 probando las 3 de arriba juntas ,empeoran el modelo
# ✅ MAE con weather lags 24 y 48h: 67.3127
# 📊 MAE anterior: 62.9796

# Modelo #32 agrego fdirect_solar_radiation_lag24h
# ✅ MAE con weather lags 24h : 61.5811
# 📊 MAE anterior: 62.9796

# Modelo #33 pruebo ftemperature_lag48h y fcloudcover_total_lag48h
# ✅ MAE con weather lags 24 y 48h: 62.3582
# 📊 MAE anterior: 62.9796

# Modelo #34 pruebo fdirect_solar_radiation_lag24h y ftemperature_lag48h
# ✅ MAE con weather lags 24 y 48h: 64.1737
# 📊 MAE anterior: 62.9796

# Modelo #35 pruebo fdirect_solar_radiation_lag24h y fcloudcover_total_lag48h me quedo conn el modelo #32
# ✅ MAE con weather lags 24 y 48h: 62.3281
# 📊 MAE anterior: 62.9796
 
# agrego fdirect_solar_radiation_lag24h a train7 pasa a ser train8 


# Modelo #36 
# fdewpoint_lag24h: 64.4357 (+2.8546) ❌ EMPEORA
# fdewpoint_lag48h: 67.4056 (+5.8245) ❌ EMPEORA
# fcloudcover_high_lag24h: 66.1002 (+4.5191) ❌ EMPEORA
# fcloudcover_high_lag48h: 66.3328 (+4.7517) ❌ EMPEORA
# fcloudcover_low_lag24h: 62.4589 (+0.8778) ❌ EMPEORA
# fcloudcover_low_lag48h: 66.0350 (+4.4539) ❌ EMPEORA
# fcloudcover_mid_lag24h: 64.7723 (+3.1912) ❌ EMPEORA
# fcloudcover_mid_lag48h: 67.0185 (+5.4374) ❌ EMPEORA
# f10_metre_u_wind_component_lag24h: 63.2744 (+1.6933) ❌ EMPEORA
# f10_metre_u_wind_component_lag48h: 64.4923 (+2.9112) ❌ EMPEORA
# f10_metre_v_wind_component_lag24h: 68.0118 (+6.4307) ❌ EMPEORA
# f10_metre_v_wind_component_lag48h: 64.3495 (+2.7684) ❌ EMPEORA
# ftotal_precipitation_lag24h: 64.6852 (+3.1041) ❌ EMPEORA
# ftotal_precipitation_lag48h: 67.5245 (+5.9434) ❌ EMPEORA

# Modelo #37
# # temp_solar_interaction: 63.5858 (+2.0047) ❌ EMPEORA
# # cloud_solar_ratio: 64.5474 (+2.9663) ❌ EMPEORA
# # wind_magnitude: 66.2764 (+4.6953) ❌ EMPEORA
# # temp_dewpoint_diff: 65.7131 (+4.1320) ❌ EMPEORA
# # surface_direct_ratio: 65.2653 (+3.6842) ❌ EMPEORA
# # ftemperature_roll3h: 64.2013 (+2.6202) ❌ EMPEORA
# # ftemperature_roll6h: 64.3444 (+2.7633) ❌ EMPEORA
# # fdirect_solar_radiation_roll3h: 64.2987 (+2.7176) ❌ EMPEORA
# # fdirect_solar_radiation_roll6h: 66.8305 (+5.2494) ❌ EMPEORA
# # fcloudcover_total_roll3h: 64.4380 (+2.8569) ❌ EMPEORA
# # fcloudcover_total_roll6h: 65.9477 (+4.3666) ❌ EMPEORA

# # Modelo #38 la forma de calculo tiene filtracion de datos y aun asi no funciona
# # Clustering de prediction_units por comportamiento similar
# # cluster: 64.9886 (+3.4075) ❌ EMPEORA
# # cluster_hour_avg_target: 61.8005 (+0.2194) ❌ EMPEORA
# # 🔥 Probando ambas cluster features juntas...
# # cluster + cluster_hour_avg_target: 62.4489 (+0.8678) ❌ EMPEORA

# Modelo #39 long_lag_features = ['fdirect_solar_radiation_lag168h', 'fdirect_solar_radiation_lag336h', 'ftemperature_lag168h']
# fdirect_solar_radiation_lag168h: 65.1855 (+3.6044) ❌ EMPEORA
# fdirect_solar_radiation_lag336h: 65.7386 (+4.1575) ❌ EMPEORA
# ftemperature_lag168h: 63.7261 (+2.1450) ❌ EMPEORA


# Modelo #40 fdirect_solar_radiation la problematica , train8 tiene 150dias y luego corta a 90dias train8 tiene incorporada la variable
#                                                      train7 corta primero a 90dias y luego calcula , incorpora aca la variable
#✅ MAE: 64.9886 con train8
# MAE: 62.9796 con train7


# Modelo #41 pruebas con distintios dias ,parace haber concept drift

# Días	Train7 MAE	Train8 MAE	Diferencia	Ganador
# -------------------------------------------------------
# 90	61.5811		64.9886		+3.4075		TRAIN7 #modelo 31
# 100	63.9248		63.4369		-0.4878		TRAIN8
# 110	64.4976		65.3186		+0.8210		TRAIN7
# 120	65.7664		66.9320		+1.1656		TRAIN7

# Modelo 42 no ayuda y puede ser un cambio en los patrones subyacentes
# Días	Train7 MAE	Train8 MAE	Diferencia	Ganador
# -------------------------------------------------------
# 60	62.6199		63.9901		+1.3702		TRAIN7
# 70	66.6343		62.8047		-3.8296		TRAIN8
# 80	62.7194		63.1605		+0.4411		TRAIN7
# 90	61.5811		64.9886		+3.4075		TRAIN7 #modelo 31

In [59]:
# Ratio production/consumption por prediction_unit_id (histórico) #no se pudo probar
# Capacidad instalada per capita (installed_capacity / eic_count) # si mejora el modelo
# Eficiencia de paneles (production vs solar_radiation cuando is_consumption=False) # empeora el modelo
# Volatilidad de precios (rolling std de electricity/gas prices) # mejor poco 1 cosa falta probar mas
# Volatilidad individual + combinada
# electricity_price_volatility_7d = rolling(7).std() # si funciona bien
# gas_price_volatility_7d = rolling(7).std() # si funciona bien



##Temporales:

# Lag de consumo/producción del mismo prediction_unit_id (24h, 48h, 168h)# empeoran el modelo
# hour_sin y hour_cos # no va
# week_sin, week_cos # no va
# Media móvil de target por prediction_unit_id # 
# Cambios día a día (delta vs día anterior) ['temperature', 'euros_per_mwh', 'lowest_price_per_mwh', 'direct_solar_radiation']# no funciono
# Media móvil de target por prediction_unit_id lag 5 dias si y muy bien

##Weather engineering:

# weather lag #['ftemperature', 'fdirect_solar_radiation', 'fsurface_solar_radiation_downwards', 
              #'fcloudcover_total', 'fsnowfall'] pruebo estas primero con 24h y 48h
# Índice de confort térmico (combinando temp + humidity) # nop
# Potencial solar real (direct + diffuse radiation ajustado por cloudcover) #nop
# Diferencia forecast vs historical weather (para medir accuracy del forecast) # lo elimine porque el historico no esta en el test set

##Segmentación:

# Clustering de prediction_units por comportamiento similar
# Ratios county-level (agregaciones por county vs individual)




#para la red neuronal probar lo siguiente
#MLP
#DeepAR
#TFT (temporal fusion transformer)

In [60]:
#test5 = run_test_pipeline(
#    test, gas_prices_t, electricity_prices_t, client_t, 
#    forecast_weather_t, weather_station
#)